# SINDy: Sparse Identification of Nonlinear Dynamics

This notebook implements symbolic regression using the SINDy algorithm to discover governing equations from data. We explore:

1. **SINDy in Ground Truth Coordinates** - Direct application on pendulum dynamics
2. **SINDy-Autoencoder** - Learning from Cartesian coordinates when true coordinates are unknown
3. **Bonus: SINDy on Videos** - High-dimensional video data

## Mathematical Background

The pendulum equation: $\ddot{z}_t = -\sin(z_t)$

SINDy expresses the ODE as: $\ddot{z}_t = \Theta(z_t, \dot{z}_t) \cdot \Xi^*$

where $\Theta$ is a library of candidate functions and $\Xi^*$ contains the sparse coefficients.

## 1. Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from scipy.integrate import odeint
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

---
# Part 1: SINDy in Ground Truth Coordinates $z$

## 1.1 Simulation

The pendulum's equation of motion is:
$$\ddot{z}_t = -\sin(z_t)$$

For SINDy, we express this as:
$$\ddot{z}_t = \Theta(z_t, \dot{z}_t) \cdot \Xi^* = \sin(z_t) \cdot (-1.0)$$

## 1.2 Implementation & Training

### Define the SINDy Library

The library of candidate functions:
$$\Theta(z, \dot{z}) = [1, z, \dot{z}, \sin(z), z^2, z \cdot \dot{z}, \dot{z} \cdot \sin(z), \dot{z}^2, \dot{z} \cdot \sin(z), \sin(z)^2]$$

In [ ]:
# Define the library of candidate functions
def get_library_terms():
    """
    Returns a list of functions representing the SINDy library terms.
    Each function takes (z, dz) and returns the term value.
    
    Library: [1, z, dz, sin(z), z², z·dz, dz·sin(z), dz², dz·sin(z), sin(z)²]
    Note: The exercise lists dz·sin(z) twice, we'll keep 10 terms as specified.
    """
    terms = [
        lambda z, dz: np.ones_like(z),           # 1
        lambda z, dz: z,                          # z
        lambda z, dz: dz,                         # dz
        lambda z, dz: np.sin(z),                  # sin(z)
        lambda z, dz: z**2,                       # z²
        lambda z, dz: z * dz,                     # z·dz
        lambda z, dz: dz * np.sin(z),             # dz·sin(z)
        lambda z, dz: dz**2,                      # dz²
        lambda z, dz: dz * np.sin(z),             # dz·sin(z) (duplicate as per spec)
        lambda z, dz: np.sin(z)**2,               # sin(z)²
    ]
    return terms

def get_term_names():
    """Returns human-readable names for each library term."""
    return ['1', 'z', 'dz', 'sin(z)', 'z²', 'z·dz', 'dz·sin(z)', 'dz²', 'dz·sin(z)', 'sin(z)²']

# Ground truth coefficients: only sin(z) term has coefficient -1
GROUND_TRUTH_COEFFICIENTS = np.array([0., 0., 0., -1., 0., 0., 0., 0., 0., 0.])

print("Library terms:", get_term_names())
print("Ground truth coefficients:", GROUND_TRUTH_COEFFICIENTS)

### Pendulum ODE Functions

In [ ]:
def pendulum_rhs(zt, dzt, coefficients, terms):
    """
    Compute scalar product Θ(z, dz) · Ξ at given time points.
    
    Parameters:
    -----------
    zt : array-like
        Angle values (can be vector)
    dzt : array-like
        Angular velocity values (can be vector)
    coefficients : array-like
        Coefficient vector Ξ
    terms : list
        List of library term functions
    
    Returns:
    --------
    ddzt : array-like
        Second derivative (acceleration)
    """
    zt = np.atleast_1d(zt)
    dzt = np.atleast_1d(dzt)
    
    # Build library matrix Θ
    theta = np.column_stack([term(zt, dzt) for term in terms])
    
    # Compute Θ · Ξ
    ddzt = theta @ coefficients
    
    return ddzt


def pendulum_ode_step(y, t, coefficients, terms):
    """
    Function for use with scipy.integrate.odeint.
    
    Parameters:
    -----------
    y : array-like
        State vector [z, dz]
    t : float
        Time (not used, but required by odeint)
    coefficients : array-like
        Coefficient vector Ξ
    terms : list
        List of library term functions
    
    Returns:
    --------
    dydt : array-like
        Derivative [dz, ddz]
    """
    z, dz = y
    ddz = pendulum_rhs(z, dz, coefficients, terms)
    return [dz, float(ddz)]


def simulate_pendulum(z0, dz0, coefficients, terms, T, dt):
    """
    Simulate pendulum using odeint.
    
    Parameters:
    -----------
    z0 : float
        Initial angle
    dz0 : float
        Initial angular velocity
    coefficients : array-like
        Coefficient vector Ξ
    terms : list
        List of library term functions
    T : int
        Number of timesteps
    dt : float
        Time step size
    
    Returns:
    --------
    t : array
        Time array
    z : array
        Angle trajectory
    dz : array
        Angular velocity trajectory
    ddz : array
        Angular acceleration trajectory
    """
    t = np.arange(T) * dt
    y0 = [z0, dz0]
    
    # Integrate ODE
    solution = odeint(pendulum_ode_step, y0, t, args=(coefficients, terms))
    
    z = solution[:, 0]
    dz = solution[:, 1]
    
    # Compute ddz from the equation
    ddz = pendulum_rhs(z, dz, coefficients, terms)
    
    return t, z, dz, ddz


# Test with ground truth coefficients
terms = get_library_terms()
t_test, z_test, dz_test, ddz_test = simulate_pendulum(
    z0=0.5, dz0=0.0, 
    coefficients=GROUND_TRUTH_COEFFICIENTS, 
    terms=terms, 
    T=100, dt=0.02
)

print(f"Simulation test: T={len(t_test)}, z range=[{z_test.min():.3f}, {z_test.max():.3f}]")

### Create Training Dataset

In [ ]:
def create_pendulum_data(z0_min, z0_max, dz0_min, dz0_max, coefficients, terms, 
                         T, dt, N, embedding=None, rejection=True):
    """
    Create training set of N simulations from uniform random initial conditions.
    
    Parameters:
    -----------
    z0_min, z0_max : float
        Range for initial angle
    dz0_min, dz0_max : float
        Range for initial angular velocity
    coefficients : array-like
        Coefficient vector Ξ
    terms : list
        List of library term functions
    T : int
        Number of timesteps per simulation
    dt : float
        Time step size
    N : int
        Number of simulations
    embedding : str or None
        'cartesian', 'grid', or None for raw coordinates
    rejection : bool
        If True, reject high angular momentum initial conditions
    
    Returns:
    --------
    data : dict
        Dictionary containing z, dz, ddz, t, and optionally embedded coordinates
    """
    all_z = []
    all_dz = []
    all_ddz = []
    all_t = []
    
    n_accepted = 0
    n_rejected = 0
    
    while n_accepted < N:
        # Sample random initial conditions
        z0 = np.random.uniform(z0_min, z0_max)
        dz0 = np.random.uniform(dz0_min, dz0_max)
        
        # Check rejection criterion: |½dz0² - cos(z0)| > 0.99
        if rejection:
            energy = np.abs(0.5 * dz0**2 - np.cos(z0))
            if energy > 0.99:
                n_rejected += 1
                continue
        
        # Simulate
        t, z, dz, ddz = simulate_pendulum(z0, dz0, coefficients, terms, T, dt)
        
        all_z.append(z)
        all_dz.append(dz)
        all_ddz.append(ddz)
        all_t.append(t)
        n_accepted += 1
    
    if rejection and n_rejected > 0:
        print(f"Rejection sampling: accepted {n_accepted}, rejected {n_rejected}")
    
    # Stack into arrays
    z_arr = np.array(all_z)      # Shape: (N, T)
    dz_arr = np.array(all_dz)
    ddz_arr = np.array(all_ddz)
    t_arr = np.array(all_t)
    
    data = {
        'z': z_arr,
        'dz': dz_arr,
        'ddz': ddz_arr,
        't': t_arr,
        'N': N,
        'T': T,
        'dt': dt
    }
    
    # Apply embedding if specified
    if embedding == 'cartesian':
        x, dx, ddx = embed_cartesian(z_arr, dz_arr, ddz_arr)
        data['x'] = x
        data['dx'] = dx
        data['ddx'] = ddx
    elif embedding == 'grid':
        # Will be implemented later
        pass
    
    return data


# Create training data with recommended parameters
N = 100
z0_min, z0_max = -np.pi, np.pi
dz0_min, dz0_max = -2.1, 2.1
T = 50
dt = 0.02

terms = get_library_terms()

print("Creating training data...")
train_data = create_pendulum_data(
    z0_min=z0_min, z0_max=z0_max,
    dz0_min=dz0_min, dz0_max=dz0_max,
    coefficients=GROUND_TRUTH_COEFFICIENTS,
    terms=terms,
    T=T, dt=dt, N=N,
    embedding=None,
    rejection=True
)

print(f"\nTraining set size: {N} simulations × {T} timesteps = {N*T} samples")
print(f"z shape: {train_data['z'].shape}")
print(f"dz shape: {train_data['dz'].shape}")
print(f"ddz shape: {train_data['ddz'].shape}")

### Visualize Simulated Data

Select 5 simulations and plot $z_t$, $\dot{z}_t$, $\ddot{z}_t$ versus time.

In [ ]:
# Visualize 5 random simulations
fig, axes = plt.subplots(3, 5, figsize=(15, 8))

selected_indices = np.random.choice(N, 5, replace=False)

for col, idx in enumerate(selected_indices):
    t = train_data['t'][idx]
    z = train_data['z'][idx]
    dz = train_data['dz'][idx]
    ddz = train_data['ddz'][idx]
    
    axes[0, col].plot(t, z, 'b-')
    axes[0, col].set_ylabel('$z_t$' if col == 0 else '')
    axes[0, col].set_title(f'Simulation {idx}')
    axes[0, col].grid(True, alpha=0.3)
    
    axes[1, col].plot(t, dz, 'g-')
    axes[1, col].set_ylabel('$\dot{z}_t$' if col == 0 else '')
    axes[1, col].grid(True, alpha=0.3)
    
    axes[2, col].plot(t, ddz, 'r-')
    axes[2, col].set_ylabel('$\ddot{z}_t$' if col == 0 else '')
    axes[2, col].set_xlabel('Time (s)')
    axes[2, col].grid(True, alpha=0.3)

plt.suptitle('Pendulum Dynamics: $z$, $\dot{z}$, $\ddot{z}$ vs Time', fontsize=14)
plt.tight_layout()
plt.show()

print("Curves should oscillate and be shifted relative to each other.")

### Optional: Animate Pendulum

Animate pendulum motion with tip coordinates $x_1(t) = \sin(z_t)$, $x_2(t) = -\cos(z_t)$.

In [ ]:
def animate_pendulum(z, dt, save_path=None):
    """
    Animate pendulum motion.
    
    Parameters:
    -----------
    z : array
        Angle trajectory
    dt : float
        Time step
    save_path : str or None
        Path to save animation
    
    Returns:
    --------
    anim : FuncAnimation object
    """
    # Compute tip positions
    x1 = np.sin(z)
    x2 = -np.cos(z)
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.set_title('Pendulum Animation')
    
    # Draw pivot
    pivot, = ax.plot([0], [0], 'ko', markersize=10)
    
    # Initialize line and bob
    line, = ax.plot([], [], 'b-', linewidth=2)
    bob, = ax.plot([], [], 'ro', markersize=15)
    trail, = ax.plot([], [], 'r-', alpha=0.3, linewidth=1)
    
    trail_x, trail_y = [], []
    
    def init():
        line.set_data([], [])
        bob.set_data([], [])
        trail.set_data([], [])
        return line, bob, trail
    
    def animate(i):
        line.set_data([0, x1[i]], [0, x2[i]])
        bob.set_data([x1[i]], [x2[i]])
        
        trail_x.append(x1[i])
        trail_y.append(x2[i])
        trail.set_data(trail_x, trail_y)
        
        return line, bob, trail
    
    anim = FuncAnimation(fig, animate, init_func=init, 
                        frames=len(z), interval=dt*1000, blit=True)
    
    plt.close(fig)
    return anim


# Create animation for one simulation
anim = animate_pendulum(train_data['z'][0], dt)
HTML(anim.to_jshtml())

### SINDy with Sklearn LASSO

Implement LASSO regression to find sparse coefficients:
$$\hat{\Xi} = \arg\min_{\Xi}\left[\frac{1}{T \cdot N}\sum_{i=1}^{T \cdot N}\left|\ddot{z}^i - \Theta(z^i, \dot{z}^i) \cdot \Xi\right|_2^2 + \lambda |\Xi|_1\right]$$

In [ ]:
def build_library_matrix(z, dz, terms):
    """
    Build the library matrix Θ(z, dz).
    
    Parameters:
    -----------
    z : array-like
        Flattened angle values
    dz : array-like
        Flattened angular velocity values
    terms : list
        List of library term functions
    
    Returns:
    --------
    Theta : ndarray
        Library matrix of shape (n_samples, n_terms)
    """
    z = np.atleast_1d(z).flatten()
    dz = np.atleast_1d(dz).flatten()
    
    Theta = np.column_stack([term(z, dz) for term in terms])
    return Theta


def sindy_sklearn(z, dz, ddz, terms, alpha=1e-5):
    """
    Train SINDy using sklearn LASSO.
    
    Parameters:
    -----------
    z, dz, ddz : ndarray
        Training data (can be 2D with shape (N, T))
    terms : list
        List of library term functions
    alpha : float
        L1 regularization strength
    
    Returns:
    --------
    coefficients : ndarray
        Learned coefficient vector
    model : Lasso
        Trained sklearn model
    """
    # Flatten arrays
    z_flat = z.flatten()
    dz_flat = dz.flatten()
    ddz_flat = ddz.flatten()
    
    # Build library matrix
    Theta = build_library_matrix(z_flat, dz_flat, terms)
    
    # Fit LASSO
    model = Lasso(alpha=alpha, fit_intercept=False, max_iter=10000)
    model.fit(Theta, ddz_flat)
    
    return model.coef_, model


# Train sklearn SINDy
terms = get_library_terms()
term_names = get_term_names()

sklearn_coeffs, sklearn_model = sindy_sklearn(
    train_data['z'], train_data['dz'], train_data['ddz'], 
    terms, alpha=1e-5
)

print("Sklearn LASSO Results:")
print("-" * 40)
for name, coef, gt in zip(term_names, sklearn_coeffs, GROUND_TRUTH_COEFFICIENTS):
    marker = "✓" if np.abs(coef - gt) < 0.1 else ""
    print(f"{name:15s}: {coef:10.6f} (GT: {gt:6.2f}) {marker}")

print(f"\nMSE: {np.mean((sklearn_coeffs - GROUND_TRUTH_COEFFICIENTS)**2):.2e}")

### SINDy with PyTorch

Implement SINDy as a PyTorch module with learnable coefficients and boolean mask.

In [ ]:
def build_library_matrix_torch(z, dz):
    """
    Build the library matrix Θ(z, dz) using PyTorch tensors.
    
    Parameters:
    -----------
    z : torch.Tensor
        Angle values
    dz : torch.Tensor
        Angular velocity values
    
    Returns:
    --------
    Theta : torch.Tensor
        Library matrix of shape (n_samples, n_terms)
    """
    z = z.view(-1)
    dz = dz.view(-1)
    
    sin_z = torch.sin(z)
    
    Theta = torch.stack([
        torch.ones_like(z),      # 1
        z,                        # z
        dz,                       # dz
        sin_z,                    # sin(z)
        z**2,                     # z²
        z * dz,                   # z·dz
        dz * sin_z,               # dz·sin(z)
        dz**2,                    # dz²
        dz * sin_z,               # dz·sin(z)
        sin_z**2,                 # sin(z)²
    ], dim=1)
    
    return Theta


class SINDy(nn.Module):
    """
    SINDy module with learnable coefficients and boolean mask.
    """
    def __init__(self, n_terms=10):
        super(SINDy, self).__init__()
        
        # Learnable coefficients
        self.coefficients = nn.Parameter(torch.ones(n_terms))
        
        # Boolean mask (not a parameter, but tracked)
        self.register_buffer('mask', torch.ones(n_terms, dtype=torch.bool))
        
        self.n_terms = n_terms
    
    def forward(self, z, dz):
        """
        Compute RHS: Θ(z, dz) · (Ξ ⊙ Υ)
        
        Parameters:
        -----------
        z : torch.Tensor
            Angle values
        dz : torch.Tensor
            Angular velocity values
        
        Returns:
        --------
        ddz : torch.Tensor
            Predicted second derivative
        """
        Theta = build_library_matrix_torch(z, dz)
        
        # Apply mask to coefficients
        masked_coeffs = self.coefficients * self.mask.float()
        
        # Compute Θ · Ξ
        ddz = Theta @ masked_coeffs
        
        return ddz
    
    def get_coefficients(self):
        """Return masked coefficients as numpy array."""
        return (self.coefficients * self.mask.float()).detach().cpu().numpy()
    
    def get_mask(self):
        """Return mask as numpy array."""
        return self.mask.detach().cpu().numpy()
    
    def set_mask(self, mask):
        """Set the mask."""
        self.mask.copy_(torch.tensor(mask, dtype=torch.bool))
    
    def l1_regularization(self):
        """Compute L1 norm of masked coefficients."""
        return torch.sum(torch.abs(self.coefficients * self.mask.float()))


# Test SINDy module
sindy_model = SINDy(n_terms=10).to(device)
z_test = torch.randn(100).to(device)
dz_test = torch.randn(100).to(device)
ddz_pred = sindy_model(z_test, dz_test)
print(f"SINDy model test: input shape {z_test.shape}, output shape {ddz_pred.shape}")

### Training Function with Thresholding

Implement `train_sindy()` with optional Sequential Thresholding (ST) and Patient Trend-Aware Thresholding (PTAT).

In [ ]:
def train_sindy(model, train_loader, val_loader, 
                n_epochs=1000, lr=1e-3, lambda_l1=1e-5,
                thresholding=None, threshold_a=0.1, threshold_b=0.002,
                threshold_interval=500, patience=1000,
                verbose=True):
    """
    Train SINDy model with optional thresholding.
    
    Parameters:
    -----------
    model : SINDy
        SINDy model to train
    train_loader : DataLoader
        Training data loader
    val_loader : DataLoader
        Validation data loader
    n_epochs : int
        Number of training epochs
    lr : float
        Learning rate
    lambda_l1 : float
        L1 regularization strength
    thresholding : str or None
        'sequential', 'patient', or None
    threshold_a : float
        Coefficient magnitude threshold
    threshold_b : float
        Coefficient change threshold (for PTAT)
    threshold_interval : int
        Epochs between thresholding (for ST)
    patience : int
        Patience for PTAT
    verbose : bool
        Print progress
    
    Returns:
    --------
    history : dict
        Training history with losses and coefficients
    """
    optimizer = optim.Adam(model.parameters(), lr=lr)
    mse_loss = nn.MSELoss()
    
    history = {
        'train_loss': [],
        'val_loss': [],
        'coefficients': [],
        'mask': []
    }
    
    # For PTAT
    if thresholding == 'patient':
        xi_prev = torch.zeros_like(model.coefficients)
        E_a = torch.zeros(model.n_terms, device=device)  # Last epoch where |Ξ| > a
        E_b = torch.zeros(model.n_terms, device=device)  # Last epoch where |Ξ - Ξ_prev| > b
    
    for epoch in range(n_epochs):
        # Training
        model.train()
        train_losses = []
        
        for z_batch, dz_batch, ddz_batch in train_loader:
            z_batch = z_batch.to(device)
            dz_batch = dz_batch.to(device)
            ddz_batch = ddz_batch.to(device)
            
            optimizer.zero_grad()
            
            ddz_pred = model(z_batch, dz_batch)
            loss = mse_loss(ddz_pred, ddz_batch.view(-1))
            
            # Add L1 regularization
            loss += lambda_l1 * model.l1_regularization()
            
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
        
        # Validation
        model.eval()
        val_losses = []
        
        with torch.no_grad():
            for z_batch, dz_batch, ddz_batch in val_loader:
                z_batch = z_batch.to(device)
                dz_batch = dz_batch.to(device)
                ddz_batch = ddz_batch.to(device)
                
                ddz_pred = model(z_batch, dz_batch)
                loss = mse_loss(ddz_pred, ddz_batch.view(-1))
                val_losses.append(loss.item())
        
        avg_train_loss = np.mean(train_losses)
        avg_val_loss = np.mean(val_losses)
        
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['coefficients'].append(model.get_coefficients().copy())
        history['mask'].append(model.get_mask().copy())
        
        # Apply thresholding
        if thresholding == 'sequential':
            # Sequential Thresholding (ST)
            if (epoch + 1) % threshold_interval == 0:
                with torch.no_grad():
                    coeffs = model.coefficients.abs()
                    mask = model.mask.clone()
                    
                    # Set small coefficients to 0 and update mask
                    small_coeffs = coeffs < threshold_a
                    model.coefficients.data[small_coeffs] = 0
                    mask[small_coeffs] = False
                    model.mask.copy_(mask)
                    
                    if verbose:
                        n_active = mask.sum().item()
                        print(f"Epoch {epoch+1}: Thresholded, {n_active} active terms")
        
        elif thresholding == 'patient':
            # Patient Trend-Aware Thresholding (PTAT)
            with torch.no_grad():
                coeffs = model.coefficients.clone()
                
                # Update E_a: epochs where |Ξ| > a
                over_a = coeffs.abs() > threshold_a
                E_a[over_a] = epoch + 1
                
                # Update E_b: epochs where |Ξ - Ξ_prev| > b
                delta = (coeffs - xi_prev).abs() > threshold_b
                E_b[delta] = epoch + 1
                
                # Compute temporary mask
                within_patience_a = (epoch + 1 - E_a) < patience
                within_patience_b = (epoch + 1 - E_b) < patience
                temp_mask = within_patience_a | within_patience_b
                
                # Update mask (can only turn off, not on)
                new_mask = model.mask & temp_mask
                model.mask.copy_(new_mask)
                
                # Zero out masked coefficients
                model.coefficients.data[~model.mask] = 0
                
                xi_prev = coeffs.clone()
        
        # Print progress
        if verbose and (epoch + 1) % 100 == 0:
            n_active = model.mask.sum().item()
            print(f"Epoch {epoch+1}/{n_epochs}: Train Loss = {avg_train_loss:.6f}, "
                  f"Val Loss = {avg_val_loss:.6f}, Active terms = {n_active}")
    
    return history


# Prepare data loaders
z_flat = train_data['z'].flatten()
dz_flat = train_data['dz'].flatten()
ddz_flat = train_data['ddz'].flatten()

# Split into train/val
z_train, z_val, dz_train, dz_val, ddz_train, ddz_val = train_test_split(
    z_flat, dz_flat, ddz_flat, test_size=0.2, random_state=42
)

# Create tensors
train_dataset = TensorDataset(
    torch.tensor(z_train, dtype=torch.float32),
    torch.tensor(dz_train, dtype=torch.float32),
    torch.tensor(ddz_train, dtype=torch.float32)
)
val_dataset = TensorDataset(
    torch.tensor(z_val, dtype=torch.float32),
    torch.tensor(dz_val, dtype=torch.float32),
    torch.tensor(ddz_val, dtype=torch.float32)
)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")

### Train PyTorch SINDy (No Thresholding)

In [ ]:
# Train PyTorch SINDy without thresholding
print("Training PyTorch SINDy (no thresholding)...")
pytorch_sindy = SINDy(n_terms=10).to(device)

history_no_thresh = train_sindy(
    pytorch_sindy, train_loader, val_loader,
    n_epochs=1000, lr=1e-3, lambda_l1=1e-5,
    thresholding=None,
    verbose=True
)

pytorch_coeffs = pytorch_sindy.get_coefficients()

print("\nPyTorch SINDy Results (No Thresholding):")
print("-" * 40)
for name, coef, gt in zip(term_names, pytorch_coeffs, GROUND_TRUTH_COEFFICIENTS):
    marker = "✓" if np.abs(coef - gt) < 0.1 else ""
    print(f"{name:15s}: {coef:10.6f} (GT: {gt:6.2f}) {marker}")

### Compare Sklearn vs PyTorch Solutions

In [ ]:
# Compare sklearn and pytorch solutions
print("Comparison of Sklearn vs PyTorch Solutions:")
print("="*60)

print(f"\n{'Term':15s} {'Sklearn':>12s} {'PyTorch':>12s} {'GT':>8s}")
print("-"*50)
for name, sk_coef, pt_coef, gt in zip(term_names, sklearn_coeffs, pytorch_coeffs, GROUND_TRUTH_COEFFICIENTS):
    print(f"{name:15s} {sk_coef:12.6f} {pt_coef:12.6f} {gt:8.2f}")

diff = np.max(np.abs(sklearn_coeffs - pytorch_coeffs))
print(f"\nMax difference between sklearn and pytorch: {diff:.6e}")

if diff < 0.01:
    print("✓ Solutions match within numerical accuracy!")
else:
    print("⚠ Solutions differ - this may be due to different regularization paths")

# Visualize comparison
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(len(term_names))
width = 0.25

bars1 = ax.bar(x - width, sklearn_coeffs, width, label='Sklearn LASSO', alpha=0.8)
bars2 = ax.bar(x, pytorch_coeffs, width, label='PyTorch (no thresh)', alpha=0.8)
bars3 = ax.bar(x + width, GROUND_TRUTH_COEFFICIENTS, width, label='Ground Truth', alpha=0.8)

ax.set_xlabel('Term')
ax.set_ylabel('Coefficient')
ax.set_title('Comparison: Sklearn vs PyTorch vs Ground Truth')
ax.set_xticks(x)
ax.set_xticklabels(term_names, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Compare sklearn and pytorch solutions
print("Comparison: Sklearn vs PyTorch")
print("=" * 60)
print(f"{'Term':15s} {'Sklearn':>12s} {'PyTorch':>12s} {'Ground Truth':>12s}")
print("-" * 60)
for name, sk_c, pt_c, gt in zip(term_names, sklearn_coeffs, pytorch_coeffs, GROUND_TRUTH_COEFFICIENTS):
    print(f"{name:15s} {sk_c:12.6f} {pt_c:12.6f} {gt:12.2f}")

print("-" * 60)
print(f"Max difference between Sklearn and PyTorch: {np.max(np.abs(sklearn_coeffs - pytorch_coeffs)):.6f}")

# Plot coefficient comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(len(term_names))
width = 0.25

# Coefficient comparison
axes[0].bar(x - width, sklearn_coeffs, width, label='Sklearn', alpha=0.8)
axes[0].bar(x, pytorch_coeffs, width, label='PyTorch', alpha=0.8)
axes[0].bar(x + width, GROUND_TRUTH_COEFFICIENTS, width, label='Ground Truth', alpha=0.8)
axes[0].set_xticks(x)
axes[0].set_xticklabels(term_names, rotation=45, ha='right')
axes[0].set_ylabel('Coefficient Value')
axes[0].set_title('Learned Coefficients Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)

# Training history
axes[1].semilogy(history_no_thresh['train_loss'], label='Train Loss')
axes[1].semilogy(history_no_thresh['val_loss'], label='Val Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss (log scale)')
axes[1].set_title('Training History (PyTorch)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 1.3 Thresholding

### Train with Sequential Thresholding (ST)

In [ ]:
# Train with Sequential Thresholding
print("Training PyTorch SINDy with Sequential Thresholding (ST)...")
pytorch_sindy_st = SINDy(n_terms=10).to(device)

history_st = train_sindy(
    pytorch_sindy_st, train_loader, val_loader,
    n_epochs=2000, lr=1e-3, lambda_l1=1e-5,
    thresholding='sequential',
    threshold_a=0.1, threshold_interval=500,
    verbose=True
)

st_coeffs = pytorch_sindy_st.get_coefficients()
st_mask = pytorch_sindy_st.get_mask()

print("\nSequential Thresholding Results:")
print("-" * 50)
for name, coef, mask, gt in zip(term_names, st_coeffs, st_mask, GROUND_TRUTH_COEFFICIENTS):
    status = "Active" if mask else "Inactive"
    marker = "✓" if np.abs(coef - gt) < 0.1 else ""
    print(f"{name:15s}: {coef:10.6f} [{status:8s}] (GT: {gt:6.2f}) {marker}")

### Train with Patient Trend-Aware Thresholding (PTAT)

In [ ]:
# Train with Patient Trend-Aware Thresholding
print("Training PyTorch SINDy with Patient Trend-Aware Thresholding (PTAT)...")
pytorch_sindy_ptat = SINDy(n_terms=10).to(device)

history_ptat = train_sindy(
    pytorch_sindy_ptat, train_loader, val_loader,
    n_epochs=2000, lr=1e-3, lambda_l1=1e-5,
    thresholding='patient',
    threshold_a=0.1, threshold_b=0.002, patience=1000,
    verbose=True
)

ptat_coeffs = pytorch_sindy_ptat.get_coefficients()
ptat_mask = pytorch_sindy_ptat.get_mask()

print("\nPatient Trend-Aware Thresholding Results:")
print("-" * 50)
for name, coef, mask, gt in zip(term_names, ptat_coeffs, ptat_mask, GROUND_TRUTH_COEFFICIENTS):
    status = "Active" if mask else "Inactive"
    marker = "✓" if np.abs(coef - gt) < 0.1 else ""
    print(f"{name:15s}: {coef:10.6f} [{status:8s}] (GT: {gt:6.2f}) {marker}")

## 1.4 Evaluation & Visualization

### Visualize Coefficient History

In [ ]:
def plot_coefficient_history(history, title, term_names):
    """Plot coefficient evolution over training."""
    coeffs_history = np.array(history['coefficients'])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot all coefficients
    for i, name in enumerate(term_names):
        axes[0].plot(coeffs_history[:, i], label=name, alpha=0.8)
    
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Coefficient Value')
    axes[0].set_title(f'{title} - Coefficient Evolution')
    axes[0].legend(loc='upper right', fontsize=8)
    axes[0].grid(True, alpha=0.3)
    axes[0].axhline(y=-1, color='k', linestyle='--', label='Ground Truth sin(z)')
    
    # Plot loss
    axes[1].semilogy(history['train_loss'], label='Train Loss')
    axes[1].semilogy(history['val_loss'], label='Val Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss (log scale)')
    axes[1].set_title(f'{title} - Training Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# Plot histories
plot_coefficient_history(history_no_thresh, "No Thresholding", term_names)
plot_coefficient_history(history_st, "Sequential Thresholding", term_names)
plot_coefficient_history(history_ptat, "Patient Trend-Aware Thresholding", term_names)

### Resimulate with Learned Equations

Test the learned equations by resimulating with test initial conditions.

In [ ]:
# Resimulate with learned coefficients and compare to ground truth
def evaluate_resimulation(coefficients, terms, test_z0, test_dz0, T, dt):
    """
    Resimulate with learned coefficients and compute error.
    """
    # Ground truth simulation
    _, z_gt, dz_gt, ddz_gt = simulate_pendulum(
        test_z0, test_dz0, GROUND_TRUTH_COEFFICIENTS, terms, T, dt
    )
    
    # Learned simulation
    t, z_learned, dz_learned, ddz_learned = simulate_pendulum(
        test_z0, test_dz0, coefficients, terms, T, dt
    )
    
    # Compute errors
    z_error = np.abs(z_learned - z_gt)
    dz_error = np.abs(dz_learned - dz_gt)
    
    return t, z_gt, z_learned, z_error, dz_error


# Test with different initial conditions
test_initial_conditions = [
    (0.5, 0.0),
    (1.0, 0.5),
    (2.0, -0.5),
]

fig, axes = plt.subplots(len(test_initial_conditions), 3, figsize=(15, 4*len(test_initial_conditions)))

for row, (z0, dz0) in enumerate(test_initial_conditions):
    t, z_gt, z_st, z_err_st, _ = evaluate_resimulation(st_coeffs, terms, z0, dz0, 100, 0.02)
    _, _, z_ptat, z_err_ptat, _ = evaluate_resimulation(ptat_coeffs, terms, z0, dz0, 100, 0.02)
    
    # Plot z trajectories
    axes[row, 0].plot(t, z_gt, 'k-', label='Ground Truth', linewidth=2)
    axes[row, 0].plot(t, z_st, 'b--', label='ST', alpha=0.8)
    axes[row, 0].plot(t, z_ptat, 'r-.', label='PTAT', alpha=0.8)
    axes[row, 0].set_xlabel('Time (s)')
    axes[row, 0].set_ylabel('$z(t)$')
    axes[row, 0].set_title(f'Trajectory: $z_0={z0}$, $\dot{{z}}_0={dz0}$')
    axes[row, 0].legend()
    axes[row, 0].grid(True, alpha=0.3)
    
    # Plot errors
    axes[row, 1].semilogy(t, z_err_st, 'b-', label='ST Error')
    axes[row, 1].semilogy(t, z_err_ptat, 'r-', label='PTAT Error')
    axes[row, 1].set_xlabel('Time (s)')
    axes[row, 1].set_ylabel('$|z - \hat{z}|$')
    axes[row, 1].set_title('Error over Time')
    axes[row, 1].legend()
    axes[row, 1].grid(True, alpha=0.3)
    
    # Plot phase portrait
    axes[row, 2].plot(z_gt, np.gradient(z_gt, t), 'k-', label='Ground Truth', linewidth=2)
    axes[row, 2].plot(z_st, np.gradient(z_st, t), 'b--', label='ST', alpha=0.8)
    axes[row, 2].plot(z_ptat, np.gradient(z_ptat, t), 'r-.', label='PTAT', alpha=0.8)
    axes[row, 2].set_xlabel('$z$')
    axes[row, 2].set_ylabel('$\dot{z}$')
    axes[row, 2].set_title('Phase Portrait')
    axes[row, 2].legend()
    axes[row, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compute average error over time for multiple test cases
n_test = 20
T_test = 50
all_errors_st = []
all_errors_ptat = []

for _ in range(n_test):
    z0 = np.random.uniform(-np.pi, np.pi)
    dz0 = np.random.uniform(-2.0, 2.0)
    
    t, _, _, z_err_st, _ = evaluate_resimulation(st_coeffs, terms, z0, dz0, T_test, dt)
    _, _, _, z_err_ptat, _ = evaluate_resimulation(ptat_coeffs, terms, z0, dz0, T_test, dt)
    
    all_errors_st.append(z_err_st)
    all_errors_ptat.append(z_err_ptat)

avg_error_st = np.mean(all_errors_st, axis=0)
avg_error_ptat = np.mean(all_errors_ptat, axis=0)

plt.figure(figsize=(10, 5))
plt.semilogy(t, avg_error_st, 'b-', label='ST (avg)', linewidth=2)
plt.semilogy(t, avg_error_ptat, 'r-', label='PTAT (avg)', linewidth=2)
plt.xlabel('Time (s)')
plt.ylabel('Average $|z - \hat{z}|$')
plt.title(f'Average Error over Time ({n_test} test cases)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 1.5 Small Angle Approximation

For small angles, $\sin(z) \approx z$. Let's train SINDy with smaller initial conditions to observe when the $z$ term becomes significant alongside or instead of $\sin(z)$.

In [ ]:
# Study small angle approximation
small_angle_ranges = [
    (0.5, 0.5),
    (0.3, 0.3),
    (0.1, 0.1),
    (0.05, 0.05),
]

results_small_angle = []

for z0_max, dz0_max in small_angle_ranges:
    print(f"\n{'='*60}")
    print(f"Training with z0_max = {z0_max}, dz0_max = {dz0_max}")
    print('='*60)
    
    # Create data with smaller initial conditions
    small_data = create_pendulum_data(
        z0_min=-z0_max, z0_max=z0_max,
        dz0_min=-dz0_max, dz0_max=dz0_max,
        coefficients=GROUND_TRUTH_COEFFICIENTS,
        terms=terms, T=T, dt=dt, N=N,
        embedding=None, rejection=False  # No rejection for small angles
    )
    
    # Prepare data loaders
    z_flat_small = small_data['z'].flatten()
    dz_flat_small = small_data['dz'].flatten()
    ddz_flat_small = small_data['ddz'].flatten()
    
    z_tr, z_vl, dz_tr, dz_vl, ddz_tr, ddz_vl = train_test_split(
        z_flat_small, dz_flat_small, ddz_flat_small, test_size=0.2, random_state=42
    )
    
    train_ds = TensorDataset(
        torch.tensor(z_tr, dtype=torch.float32),
        torch.tensor(dz_tr, dtype=torch.float32),
        torch.tensor(ddz_tr, dtype=torch.float32)
    )
    val_ds = TensorDataset(
        torch.tensor(z_vl, dtype=torch.float32),
        torch.tensor(dz_vl, dtype=torch.float32),
        torch.tensor(ddz_vl, dtype=torch.float32)
    )
    
    train_ld = DataLoader(train_ds, batch_size=256, shuffle=True)
    val_ld = DataLoader(val_ds, batch_size=256, shuffle=False)
    
    # Train SINDy
    model_small = SINDy(n_terms=10).to(device)
    history_small = train_sindy(
        model_small, train_ld, val_ld,
        n_epochs=1000, lr=1e-3, lambda_l1=1e-5,
        thresholding='sequential', threshold_a=0.05, threshold_interval=500,
        verbose=False
    )
    
    coeffs_small = model_small.get_coefficients()
    mask_small = model_small.get_mask()
    
    # Print results
    print("\nLearned coefficients:")
    for name, coef, mask in zip(term_names, coeffs_small, mask_small):
        if np.abs(coef) > 0.01:  # Only show non-negligible
            status = "Active" if mask else "Inactive"
            print(f"  {name:15s}: {coef:10.6f} [{status}]")
    
    results_small_angle.append({
        'z0_max': z0_max,
        'dz0_max': dz0_max,
        'coefficients': coeffs_small,
        'mask': mask_small
    })

# Visualize how coefficients change with angle magnitude
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

z_terms = [r['coefficients'][1] for r in results_small_angle]  # z term
sin_terms = [r['coefficients'][3] for r in results_small_angle]  # sin(z) term
angle_ranges = [r['z0_max'] for r in results_small_angle]

axes[0].plot(angle_ranges, sin_terms, 'ro-', label='sin(z) coefficient', markersize=10)
axes[0].plot(angle_ranges, z_terms, 'bs-', label='z coefficient', markersize=10)
axes[0].axhline(y=-1, color='k', linestyle='--', label='Ground Truth')
axes[0].set_xlabel('Max Initial Angle (rad)')
axes[0].set_ylabel('Coefficient Value')
axes[0].set_title('Coefficients vs Initial Angle Magnitude')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].invert_xaxis()

# Compare sin(z) vs z for small angles
z_range = np.linspace(0, 0.5, 100)
axes[1].plot(z_range, np.sin(z_range), 'r-', label='sin(z)', linewidth=2)
axes[1].plot(z_range, z_range, 'b--', label='z', linewidth=2)
axes[1].plot(z_range, z_range - z_range**3/6, 'g-.', label='z - z³/6 (Taylor)', linewidth=2)
axes[1].set_xlabel('z (rad)')
axes[1].set_ylabel('Value')
axes[1].set_title('Small Angle Approximation: sin(z) ≈ z')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("EXPLANATION: Small Angle Approximation")
print("="*60)
print("""
For small angles, sin(z) ≈ z (Taylor expansion: sin(z) = z - z³/6 + ...)

When training with small initial conditions, both sin(z) and z produce 
nearly identical dynamics. This creates a collinearity issue in LASSO:
- With large angles: sin(z) is clearly distinguishable from z
- With small angles: sin(z) ≈ z, so LASSO may assign weight to either term

This is why at very small angles (z₀ < 0.1), the z term may appear with 
coefficient ≈ -1 instead of (or alongside) sin(z), as both explain the 
data equally well from the perspective of the regression objective.
""")

---
# Part 2: SINDy-Autoencoder

## 2.1 Cartesian Embedding

Now assume we don't know the canonical coordinate $z_t$. Instead, we observe the pendulum tip in 2D Cartesian coordinates:
$$x = [\sin(z), -\cos(z)]$$
$$\dot{x} = [\cos(z) \cdot \dot{z}, \sin(z) \cdot \dot{z}]$$
$$\ddot{x} = [-\sin(z) \cdot \dot{z}^2 + \cos(z) \cdot \ddot{z}, \cos(z) \cdot \dot{z}^2 + \sin(z) \cdot \ddot{z}]$$

In [ ]:
def embed_cartesian(z, dz, ddz):
    """
    Embed pendulum dynamics into Cartesian coordinates.
    
    Parameters:
    -----------
    z : ndarray
        Angle values (can be any shape)
    dz : ndarray
        Angular velocity values
    ddz : ndarray
        Angular acceleration values
    
    Returns:
    --------
    x : ndarray
        Cartesian position [sin(z), -cos(z)], shape (*z.shape, 2)
    dx : ndarray
        Cartesian velocity, shape (*z.shape, 2)
    ddx : ndarray
        Cartesian acceleration, shape (*z.shape, 2)
    """
    sin_z = np.sin(z)
    cos_z = np.cos(z)
    
    # Position: x = [sin(z), -cos(z)]
    x = np.stack([sin_z, -cos_z], axis=-1)
    
    # Velocity: dx = [cos(z)·dz, sin(z)·dz]
    dx = np.stack([cos_z * dz, sin_z * dz], axis=-1)
    
    # Acceleration: ddx = [-sin(z)·dz² + cos(z)·ddz, cos(z)·dz² + sin(z)·ddz]
    ddx = np.stack([
        -sin_z * dz**2 + cos_z * ddz,
        cos_z * dz**2 + sin_z * ddz
    ], axis=-1)
    
    return x, dx, ddx


# Create training data with Cartesian embedding
train_data_cartesian = create_pendulum_data(
    z0_min=-np.pi, z0_max=np.pi,
    dz0_min=-2.1, dz0_max=2.1,
    coefficients=GROUND_TRUTH_COEFFICIENTS,
    terms=terms, T=T, dt=dt, N=N,
    embedding='cartesian',
    rejection=True
)

# Also compute cartesian embedding manually for verification
x_cart, dx_cart, ddx_cart = embed_cartesian(
    train_data['z'], train_data['dz'], train_data['ddz']
)

print(f"Cartesian data shapes:")
print(f"  x:   {x_cart.shape}")
print(f"  dx:  {dx_cart.shape}")
print(f"  ddx: {ddx_cart.shape}")

# Visualize Cartesian embedding
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Select one trajectory
idx = 0
t_plot = train_data['t'][idx]
z_plot = train_data['z'][idx]
x_plot = x_cart[idx]
dx_plot = dx_cart[idx]
ddx_plot = ddx_cart[idx]

# Plot z trajectory
axes[0, 0].plot(t_plot, z_plot, 'b-')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('$z$')
axes[0, 0].set_title('Angle $z(t)$')
axes[0, 0].grid(True, alpha=0.3)

# Plot x1, x2
axes[0, 1].plot(t_plot, x_plot[:, 0], 'r-', label='$x_1 = \\sin(z)$')
axes[0, 1].plot(t_plot, x_plot[:, 1], 'b-', label='$x_2 = -\\cos(z)$')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Position')
axes[0, 1].set_title('Cartesian Position $x(t)$')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot trajectory in x1-x2 plane
axes[0, 2].plot(x_plot[:, 0], x_plot[:, 1], 'g-')
axes[0, 2].scatter([x_plot[0, 0]], [x_plot[0, 1]], c='r', s=100, marker='o', label='Start')
axes[0, 2].set_xlabel('$x_1$')
axes[0, 2].set_ylabel('$x_2$')
axes[0, 2].set_title('Pendulum Tip Trajectory')
axes[0, 2].set_aspect('equal')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Plot dx1, dx2
axes[1, 0].plot(t_plot, dx_plot[:, 0], 'r-', label='$\dot{x}_1$')
axes[1, 0].plot(t_plot, dx_plot[:, 1], 'b-', label='$\dot{x}_2$')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Velocity')
axes[1, 0].set_title('Cartesian Velocity $\dot{x}(t)$')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot ddx1, ddx2
axes[1, 1].plot(t_plot, ddx_plot[:, 0], 'r-', label='$\ddot{x}_1$')
axes[1, 1].plot(t_plot, ddx_plot[:, 1], 'b-', label='$\ddot{x}_2$')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Acceleration')
axes[1, 1].set_title('Cartesian Acceleration $\ddot{x}(t)$')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Phase portrait in dx-space
axes[1, 2].plot(dx_plot[:, 0], dx_plot[:, 1], 'm-')
axes[1, 2].scatter([dx_plot[0, 0]], [dx_plot[0, 1]], c='r', s=100, marker='o', label='Start')
axes[1, 2].set_xlabel('$\dot{x}_1$')
axes[1, 2].set_ylabel('$\dot{x}_2$')
axes[1, 2].set_title('Velocity Phase Portrait')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2.2 Basic Autoencoder for Hyperparameter Search

First, build a simple autoencoder to encode 2D Cartesian $x_t$ into 1D latent $z_t$.

In [ ]:
class BasicAutoencoder(nn.Module):
    """
    Simple autoencoder with Linear + Sigmoid layers.
    Encodes 2D x into 1D latent z.
    """
    def __init__(self, input_dim=2, latent_dim=1, hidden_dims=[32, 16]):
        super(BasicAutoencoder, self).__init__()
        
        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            encoder_layers.append(nn.Linear(prev_dim, h_dim))
            encoder_layers.append(nn.Sigmoid())
            prev_dim = h_dim
        encoder_layers.append(nn.Linear(prev_dim, latent_dim))
        self.encoder = nn.Sequential(*encoder_layers)
        
        # Decoder (reverse architecture)
        decoder_layers = []
        prev_dim = latent_dim
        for h_dim in reversed(hidden_dims):
            decoder_layers.append(nn.Linear(prev_dim, h_dim))
            decoder_layers.append(nn.Sigmoid())
            prev_dim = h_dim
        decoder_layers.append(nn.Linear(prev_dim, input_dim))
        self.decoder = nn.Sequential(*decoder_layers)
    
    def encode(self, x):
        return self.encoder(x)
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        z = self.encode(x)
        x_hat = self.decode(z)
        return x_hat, z


def train_autoencoder(model, train_loader, val_loader, n_epochs=500, lr=1e-3, verbose=True):
    """Train basic autoencoder."""
    optimizer = optim.Adam(model.parameters(), lr=lr)
    mse_loss = nn.MSELoss()
    
    history = {'train_loss': [], 'val_loss': []}
    
    for epoch in range(n_epochs):
        model.train()
        train_losses = []
        
        for x_batch, in train_loader:
            x_batch = x_batch.to(device)
            optimizer.zero_grad()
            x_hat, _ = model(x_batch)
            loss = mse_loss(x_hat, x_batch)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        
        model.eval()
        val_losses = []
        with torch.no_grad():
            for x_batch, in val_loader:
                x_batch = x_batch.to(device)
                x_hat, _ = model(x_batch)
                loss = mse_loss(x_hat, x_batch)
                val_losses.append(loss.item())
        
        history['train_loss'].append(np.mean(train_losses))
        history['val_loss'].append(np.mean(val_losses))
        
        if verbose and (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}: Train Loss = {history['train_loss'][-1]:.6f}, "
                  f"Val Loss = {history['val_loss'][-1]:.6f}")
    
    return history


# Prepare Cartesian data for autoencoder
x_flat = x_cart.reshape(-1, 2)
x_train, x_val = train_test_split(x_flat, test_size=0.2, random_state=42)

train_ds_ae = TensorDataset(torch.tensor(x_train, dtype=torch.float32))
val_ds_ae = TensorDataset(torch.tensor(x_val, dtype=torch.float32))

train_loader_ae = DataLoader(train_ds_ae, batch_size=256, shuffle=True)
val_loader_ae = DataLoader(val_ds_ae, batch_size=256, shuffle=False)

print(f"Autoencoder data: {len(train_ds_ae)} train, {len(val_ds_ae)} val samples")

### Hyperparameter Search

In [ ]:
# Try different hidden layer configurations
hidden_configs = [
    [8],
    [16],
    [32],
    [16, 8],
    [32, 16],
    [64, 32],
    [32, 16, 8],
]

results_hp = []

print("Hyperparameter search for autoencoder architecture...")
print("="*60)

for hidden_dims in hidden_configs:
    print(f"\nTesting hidden_dims = {hidden_dims}")
    
    model = BasicAutoencoder(input_dim=2, latent_dim=1, hidden_dims=hidden_dims).to(device)
    history = train_autoencoder(model, train_loader_ae, val_loader_ae, 
                                n_epochs=500, lr=1e-3, verbose=False)
    
    final_val_loss = history['val_loss'][-1]
    results_hp.append({
        'hidden_dims': hidden_dims,
        'val_loss': final_val_loss,
        'history': history
    })
    print(f"  Final val loss: {final_val_loss:.6f}")

# Find best configuration
best_idx = np.argmin([r['val_loss'] for r in results_hp])
best_config = results_hp[best_idx]

print("\n" + "="*60)
print(f"Best configuration: hidden_dims = {best_config['hidden_dims']}")
print(f"Best val loss: {best_config['val_loss']:.6f}")

# Visualize hyperparameter search results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot of final losses
config_names = [str(r['hidden_dims']) for r in results_hp]
val_losses = [r['val_loss'] for r in results_hp]

axes[0].bar(config_names, val_losses)
axes[0].set_xlabel('Hidden Layer Configuration')
axes[0].set_ylabel('Final Validation Loss')
axes[0].set_title('Autoencoder Architecture Comparison')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Training curves for top 3
sorted_results = sorted(results_hp, key=lambda x: x['val_loss'])[:3]
for r in sorted_results:
    axes[1].semilogy(r['history']['val_loss'], label=str(r['hidden_dims']))

axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Validation Loss')
axes[1].set_title('Training Curves (Top 3 Configurations)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2.3 Propagation of Time Derivatives

Implement layers that propagate time derivatives through the network using the chain rule.

### Sigmoid Derivatives

$$g(\tilde{z}) = \sigma(\tilde{z}) = \frac{1}{1 + e^{-\tilde{z}}}$$
$$g'(\tilde{z}) = \sigma(\tilde{z})(1 - \sigma(\tilde{z}))$$
$$g''(\tilde{z}) = \sigma'(\tilde{z})(1 - 2\sigma(\tilde{z}))$$

In [ ]:
class SigmoidDerivatives(nn.Module):
    """
    Sigmoid activation with derivative propagation.
    
    Forward pass computes:
    - z = σ(x)
    - dz = σ'(x) ⊙ dx
    - ddz = σ''(x) ⊙ dx ⊙ dx + σ'(x) ⊙ ddx
    """
    def __init__(self):
        super(SigmoidDerivatives, self).__init__()
    
    def forward(self, x, dx, ddx):
        """
        Parameters:
        -----------
        x : torch.Tensor
            Pre-activation values
        dx : torch.Tensor
            Time derivative of pre-activation
        ddx : torch.Tensor
            Second time derivative of pre-activation
        
        Returns:
        --------
        z, dz, ddz : torch.Tensor
            Output and its time derivatives
        """
        # Sigmoid and its derivatives
        sig = torch.sigmoid(x)
        sig_prime = sig * (1 - sig)
        sig_double_prime = sig_prime * (1 - 2 * sig)
        
        # Output
        z = sig
        
        # First derivative: dz = σ'(x) ⊙ dx
        dz = sig_prime * dx
        
        # Second derivative: ddz = σ''(x) ⊙ dx ⊙ dx + σ'(x) ⊙ ddx
        ddz = sig_double_prime * dx * dx + sig_prime * ddx
        
        return z, dz, ddz


class LinearDerivatives(nn.Module):
    """
    Linear layer with derivative propagation.
    
    Forward pass computes:
    - z = xW + b
    - dz = dx @ W
    - ddz = ddx @ W
    """
    def __init__(self, in_features, out_features, bias=False):
        super(LinearDerivatives, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        # Xavier uniform initialization
        nn.init.xavier_uniform_(self.linear.weight)
        if bias and self.linear.bias is not None:
            nn.init.zeros_(self.linear.bias)
    
    def forward(self, x, dx, ddx):
        """
        Parameters:
        -----------
        x : torch.Tensor
            Input values
        dx : torch.Tensor
            Time derivative of input
        ddx : torch.Tensor
            Second time derivative of input
        
        Returns:
        --------
        z, dz, ddz : torch.Tensor
            Output and its time derivatives
        """
        # Linear transformation
        z = self.linear(x)
        
        # Derivatives (just multiply by weight, no bias)
        W = self.linear.weight.t()  # Shape: (in_features, out_features)
        dz = dx @ W
        ddz = ddx @ W
        
        return z, dz, ddz
    
    @property
    def weight(self):
        return self.linear.weight


# Test derivative propagation layers
print("Testing SigmoidDerivatives...")
sig_layer = SigmoidDerivatives()
x_test = torch.randn(10, 5)
dx_test = torch.randn(10, 5)
ddx_test = torch.randn(10, 5)
z, dz, ddz = sig_layer(x_test, dx_test, ddx_test)
print(f"  Input shape: {x_test.shape}, Output shape: {z.shape}")

print("\nTesting LinearDerivatives...")
lin_layer = LinearDerivatives(5, 3)
z, dz, ddz = lin_layer(x_test, dx_test, ddx_test)
print(f"  Input shape: {x_test.shape}, Output shape: {z.shape}")

### Verify Derivative Propagation

Compare propagated derivatives with finite difference approximations:
$$\dot{z}_t \approx \frac{z_{t+1} - z_{t-1}}{2\Delta t}$$
$$\ddot{z}_t \approx \frac{z_{t-1} - 2z_t + z_{t+1}}{\Delta t^2}$$

In [ ]:
def verify_derivative_propagation(model, x, dx, ddx, dt):
    """
    Verify derivative propagation by comparing with finite differences.
    
    Parameters:
    -----------
    model : nn.Module
        Model with forward(x, dx, ddx) method
    x : ndarray
        Input sequence of shape (T, D)
    dx, ddx : ndarray
        True derivatives
    dt : float
        Time step
    
    Returns:
    --------
    errors : dict
        Dictionary with error metrics
    """
    T = x.shape[0]
    
    # Convert to tensors
    x_t = torch.tensor(x, dtype=torch.float32).to(device)
    dx_t = torch.tensor(dx, dtype=torch.float32).to(device)
    ddx_t = torch.tensor(ddx, dtype=torch.float32).to(device)
    
    model.eval()
    with torch.no_grad():
        # Get propagated derivatives
        z, dz_prop, ddz_prop = model(x_t, dx_t, ddx_t)
        
        z = z.cpu().numpy()
        dz_prop = dz_prop.cpu().numpy()
        ddz_prop = ddz_prop.cpu().numpy()
    
    # Compute finite difference derivatives
    # dz_fd[t] = (z[t+1] - z[t-1]) / (2*dt)
    # ddz_fd[t] = (z[t-1] - 2*z[t] + z[t+1]) / dt²
    
    dz_fd = np.zeros_like(z)
    ddz_fd = np.zeros_like(z)
    
    for t in range(1, T-1):
        dz_fd[t] = (z[t+1] - z[t-1]) / (2 * dt)
        ddz_fd[t] = (z[t-1] - 2*z[t] + z[t+1]) / (dt**2)
    
    # Compute errors (excluding boundary points)
    dz_error = np.mean(np.abs(dz_prop[1:-1] - dz_fd[1:-1]))
    ddz_error = np.mean(np.abs(ddz_prop[1:-1] - ddz_fd[1:-1]))
    
    return {
        'dz_prop': dz_prop,
        'dz_fd': dz_fd,
        'ddz_prop': ddz_prop,
        'ddz_fd': ddz_fd,
        'dz_error': dz_error,
        'ddz_error': ddz_error,
        'z': z
    }


# Build a simple encoder for verification
class TestEncoder(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=16, output_dim=1):
        super(TestEncoder, self).__init__()
        self.lin1 = LinearDerivatives(input_dim, hidden_dim, bias=False)
        self.sig1 = SigmoidDerivatives()
        self.lin2 = LinearDerivatives(hidden_dim, output_dim, bias=False)
    
    def forward(self, x, dx, ddx):
        z, dz, ddz = self.lin1(x, dx, ddx)
        z, dz, ddz = self.sig1(z, dz, ddz)
        z, dz, ddz = self.lin2(z, dz, ddz)
        return z, dz, ddz


# Test with actual data
test_encoder = TestEncoder(input_dim=2, hidden_dim=16, output_dim=1).to(device)

# Use one trajectory
idx = 0
x_seq = x_cart[idx]      # (T, 2)
dx_seq = dx_cart[idx]
ddx_seq = ddx_cart[idx]

verification = verify_derivative_propagation(test_encoder, x_seq, dx_seq, ddx_seq, dt)

print("Derivative Propagation Verification:")
print(f"  Mean |dz_prop - dz_fd|:   {verification['dz_error']:.6f}")
print(f"  Mean |ddz_prop - ddz_fd|: {verification['ddz_error']:.6f}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

t_plot = np.arange(len(verification['z'])) * dt

# Plot z
axes[0].plot(t_plot, verification['z'], 'b-')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('z')
axes[0].set_title('Encoded z(t)')
axes[0].grid(True, alpha=0.3)

# Plot first derivative comparison
axes[1].plot(t_plot, verification['dz_prop'], 'b-', label='Propagated')
axes[1].plot(t_plot, verification['dz_fd'], 'r--', label='Finite Diff')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('dz/dt')
axes[1].set_title('First Derivative Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot second derivative comparison
axes[2].plot(t_plot, verification['ddz_prop'], 'b-', label='Propagated')
axes[2].plot(t_plot, verification['ddz_fd'], 'r--', label='Finite Diff')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('d²z/dt²')
axes[2].set_title('Second Derivative Comparison')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNote: Small differences are expected due to finite difference approximation error.")

## 2.4 SINDy-Autoencoder Implementation

Implement `SINDyAutoencoder` containing:
- Internal SINDy instance
- Encoder/decoder with derivative propagation
- Xavier uniform initialization, no biases

In [ ]:
class SINDyAutoencoder(nn.Module):
    """
    SINDy-Autoencoder combining an autoencoder with SINDy.
    
    The encoder learns to map x -> z (latent coordinate).
    SINDy learns the dynamics ddz = Θ(z, dz) · Ξ.
    The decoder reconstructs x_hat from z.
    """
    def __init__(self, input_dim=2, latent_dim=1, encoder_hidden=[32, 16], 
                 decoder_hidden=[16, 32], n_sindy_terms=10):
        super(SINDyAutoencoder, self).__init__()
        
        # Build encoder with derivative propagation
        self.encoder_layers = nn.ModuleList()
        prev_dim = input_dim
        for h_dim in encoder_hidden:
            self.encoder_layers.append(LinearDerivatives(prev_dim, h_dim, bias=False))
            self.encoder_layers.append(SigmoidDerivatives())
            prev_dim = h_dim
        self.encoder_layers.append(LinearDerivatives(prev_dim, latent_dim, bias=False))
        
        # Build decoder with derivative propagation
        self.decoder_layers = nn.ModuleList()
        prev_dim = latent_dim
        for h_dim in decoder_hidden:
            self.decoder_layers.append(LinearDerivatives(prev_dim, h_dim, bias=False))
            self.decoder_layers.append(SigmoidDerivatives())
            prev_dim = h_dim
        self.decoder_layers.append(LinearDerivatives(prev_dim, input_dim, bias=False))
        
        # SINDy module
        self.sindy = SINDy(n_terms=n_sindy_terms)
        
        self.input_dim = input_dim
        self.latent_dim = latent_dim
    
    def encode(self, x, dx, ddx):
        """Encode x to z with derivative propagation."""
        z, dz, ddz = x, dx, ddx
        for layer in self.encoder_layers:
            z, dz, ddz = layer(z, dz, ddz)
        return z, dz, ddz
    
    def decode(self, z, dz=None, ddz=None):
        """
        Decode z to x_hat.
        If dz and ddz are provided, also propagate derivatives.
        """
        if dz is None:
            # Simple decode without derivatives (for reconstruction only)
            x = z
            for layer in self.decoder_layers:
                if isinstance(layer, LinearDerivatives):
                    x = layer.linear(x)
                elif isinstance(layer, SigmoidDerivatives):
                    x = torch.sigmoid(x)
            return x
        else:
            # Decode with derivative propagation
            x, dx, ddx = z, dz, ddz
            for layer in self.decoder_layers:
                x, dx, ddx = layer(x, dx, ddx)
            return x, dx, ddx
    
    def forward(self, x, dx, ddx):
        """
        Forward pass computing:
        - x_hat: reconstructed x
        - ddz_hat: predicted ddz from SINDy
        - ddx_hat: ddz decoded back to x-space
        
        Parameters:
        -----------
        x, dx, ddx : torch.Tensor
            Input and its time derivatives
        
        Returns:
        --------
        x_hat : torch.Tensor
            Reconstructed x
        ddz_hat : torch.Tensor
            SINDy prediction of ddz
        ddx_hat : torch.Tensor
            SINDy prediction decoded to x-space
        z : torch.Tensor
            Encoded latent variable
        dz : torch.Tensor
            Encoded latent velocity
        ddz : torch.Tensor
            Encoded latent acceleration (for loss computation)
        """
        # Encode
        z, dz, ddz_enc = self.encode(x, dx, ddx)
        
        # SINDy: predict ddz from z and dz
        ddz_hat = self.sindy(z.squeeze(-1), dz.squeeze(-1)).unsqueeze(-1)
        
        # Decode reconstruction (z only)
        x_hat = self.decode(z)
        
        # Decode SINDy prediction to x-space
        # We need to propagate the predicted ddz through the decoder
        # Using zero for dz since we're only interested in the ddx
        # Actually, we use the encoded z and dz, and replace ddz with ddz_hat
        _, _, ddx_hat = self.decode(z, dz, ddz_hat)
        
        return x_hat, ddz_hat, ddx_hat, z, dz, ddz_enc
    
    def get_sindy_coefficients(self):
        return self.sindy.get_coefficients()
    
    def get_sindy_mask(self):
        return self.sindy.get_mask()
    
    def sindy_l1_regularization(self):
        return self.sindy.l1_regularization()


# Test SINDy-Autoencoder
sindy_ae = SINDyAutoencoder(
    input_dim=2, latent_dim=1, 
    encoder_hidden=[32, 16], decoder_hidden=[16, 32],
    n_sindy_terms=10
).to(device)

# Test forward pass
x_test = torch.randn(100, 2).to(device)
dx_test = torch.randn(100, 2).to(device)
ddx_test = torch.randn(100, 2).to(device)

x_hat, ddz_hat, ddx_hat, z, dz, ddz = sindy_ae(x_test, dx_test, ddx_test)

print("SINDy-Autoencoder Test:")
print(f"  Input x shape: {x_test.shape}")
print(f"  Reconstructed x_hat shape: {x_hat.shape}")
print(f"  Latent z shape: {z.shape}")
print(f"  SINDy ddz_hat shape: {ddz_hat.shape}")
print(f"  Decoded ddx_hat shape: {ddx_hat.shape}")

### Training Function for SINDy-Autoencoder

The loss function combines:
$$\mathcal{L} = |x - \hat{x}|_2^2 + \lambda_{\ddot{z}}|\ddot{z} - \hat{\ddot{z}}|_2^2 + \lambda_{\ddot{x}}|\ddot{x} - \hat{\ddot{x}}|_2^2 + \lambda_1|\Xi|_1$$

In [ ]:
def train_sindy_autoencoder(model, train_loader, val_loader,
                           n_epochs=5000, lr=1e-3,
                           lambda_ddz=5e-5, lambda_ddx=5e-4, lambda_l1=1e-5,
                           thresholding=None, threshold_a=0.1, threshold_b=0.002,
                           threshold_interval=500, patience=1000,
                           refinement_epoch=None,
                           verbose=True, log_interval=100):
    """
    Train SINDy-Autoencoder.
    
    Parameters:
    -----------
    model : SINDyAutoencoder
        Model to train
    train_loader, val_loader : DataLoader
        Data loaders with (x, dx, ddx) tuples
    n_epochs : int
        Total training epochs
    lr : float
        Learning rate
    lambda_ddz, lambda_ddx : float
        Loss weights for latent and decoded acceleration
    lambda_l1 : float
        L1 regularization weight
    thresholding : str or None
        'sequential', 'patient', or None
    threshold_a, threshold_b : float
        Thresholding parameters
    threshold_interval : int
        For sequential thresholding
    patience : int
        For patient thresholding
    refinement_epoch : int or None
        Epoch to start refinement (λ₁ = 0)
    verbose : bool
        Print progress
    log_interval : int
        Print every log_interval epochs
    
    Returns:
    --------
    history : dict
        Training history
    """
    optimizer = optim.Adam(model.parameters(), lr=lr)
    mse_loss = nn.MSELoss()
    
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_loss_x': [],
        'train_loss_ddz': [],
        'train_loss_ddx': [],
        'val_loss_x': [],
        'val_loss_ddz': [],
        'val_loss_ddx': [],
        'coefficients': [],
        'mask': []
    }
    
    # For PTAT
    if thresholding == 'patient':
        xi_prev = torch.zeros_like(model.sindy.coefficients)
        E_a = torch.zeros(model.sindy.n_terms, device=device)
        E_b = torch.zeros(model.sindy.n_terms, device=device)
    
    current_lambda_l1 = lambda_l1
    
    for epoch in range(n_epochs):
        # Check if refinement should start
        if refinement_epoch is not None and epoch >= refinement_epoch:
            current_lambda_l1 = 0.0
        
        # Training
        model.train()
        train_losses = {'total': [], 'x': [], 'ddz': [], 'ddx': []}
        
        for x_batch, dx_batch, ddx_batch in train_loader:
            x_batch = x_batch.to(device)
            dx_batch = dx_batch.to(device)
            ddx_batch = ddx_batch.to(device)
            
            optimizer.zero_grad()
            
            x_hat, ddz_hat, ddx_hat, z, dz, ddz = model(x_batch, dx_batch, ddx_batch)
            
            # Compute losses
            loss_x = mse_loss(x_hat, x_batch)
            loss_ddz = mse_loss(ddz_hat.squeeze(), ddz.squeeze())
            loss_ddx = mse_loss(ddx_hat, ddx_batch)
            
            # Total loss
            loss = loss_x + lambda_ddz * loss_ddz + lambda_ddx * loss_ddx
            
            # Add L1 regularization
            if current_lambda_l1 > 0:
                loss += current_lambda_l1 * model.sindy_l1_regularization()
            
            loss.backward()
            optimizer.step()
            
            train_losses['total'].append(loss.item())
            train_losses['x'].append(loss_x.item())
            train_losses['ddz'].append(loss_ddz.item())
            train_losses['ddx'].append(loss_ddx.item())
        
        # Validation
        model.eval()
        val_losses = {'total': [], 'x': [], 'ddz': [], 'ddx': []}
        
        with torch.no_grad():
            for x_batch, dx_batch, ddx_batch in val_loader:
                x_batch = x_batch.to(device)
                dx_batch = dx_batch.to(device)
                ddx_batch = ddx_batch.to(device)
                
                x_hat, ddz_hat, ddx_hat, z, dz, ddz = model(x_batch, dx_batch, ddx_batch)
                
                loss_x = mse_loss(x_hat, x_batch)
                loss_ddz = mse_loss(ddz_hat.squeeze(), ddz.squeeze())
                loss_ddx = mse_loss(ddx_hat, ddx_batch)
                
                loss = loss_x + lambda_ddz * loss_ddz + lambda_ddx * loss_ddx
                
                val_losses['total'].append(loss.item())
                val_losses['x'].append(loss_x.item())
                val_losses['ddz'].append(loss_ddz.item())
                val_losses['ddx'].append(loss_ddx.item())
        
        # Record history
        history['train_loss'].append(np.mean(train_losses['total']))
        history['val_loss'].append(np.mean(val_losses['total']))
        history['train_loss_x'].append(np.mean(train_losses['x']))
        history['train_loss_ddz'].append(np.mean(train_losses['ddz']))
        history['train_loss_ddx'].append(np.mean(train_losses['ddx']))
        history['val_loss_x'].append(np.mean(val_losses['x']))
        history['val_loss_ddz'].append(np.mean(val_losses['ddz']))
        history['val_loss_ddx'].append(np.mean(val_losses['ddx']))
        history['coefficients'].append(model.get_sindy_coefficients().copy())
        history['mask'].append(model.get_sindy_mask().copy())
        
        # Apply thresholding
        if thresholding == 'sequential':
            if (epoch + 1) % threshold_interval == 0:
                with torch.no_grad():
                    coeffs = model.sindy.coefficients.abs()
                    mask = model.sindy.mask.clone()
                    
                    small_coeffs = coeffs < threshold_a
                    model.sindy.coefficients.data[small_coeffs] = 0
                    mask[small_coeffs] = False
                    model.sindy.mask.copy_(mask)
                    
                    if verbose:
                        n_active = mask.sum().item()
                        print(f"Epoch {epoch+1}: Thresholded, {n_active} active terms")
        
        elif thresholding == 'patient':
            with torch.no_grad():
                coeffs = model.sindy.coefficients.clone()
                
                over_a = coeffs.abs() > threshold_a
                E_a[over_a] = epoch + 1
                
                delta = (coeffs - xi_prev).abs() > threshold_b
                E_b[delta] = epoch + 1
                
                within_patience_a = (epoch + 1 - E_a) < patience
                within_patience_b = (epoch + 1 - E_b) < patience
                temp_mask = within_patience_a | within_patience_b
                
                new_mask = model.sindy.mask & temp_mask
                model.sindy.mask.copy_(new_mask)
                
                model.sindy.coefficients.data[~model.sindy.mask] = 0
                
                xi_prev = coeffs.clone()
        
        # Print progress
        if verbose and (epoch + 1) % log_interval == 0:
            n_active = model.sindy.mask.sum().item()
            ref_status = " [REFINEMENT]" if (refinement_epoch and epoch >= refinement_epoch) else ""
            print(f"Epoch {epoch+1}/{n_epochs}: Loss = {history['train_loss'][-1]:.6f}, "
                  f"Active = {n_active}{ref_status}")
    
    return history


# Prepare data loaders for SINDy-Autoencoder
x_ae = x_cart.reshape(-1, 2)
dx_ae = dx_cart.reshape(-1, 2)
ddx_ae = ddx_cart.reshape(-1, 2)

# Split
x_tr, x_vl, dx_tr, dx_vl, ddx_tr, ddx_vl = train_test_split(
    x_ae, dx_ae, ddx_ae, test_size=0.2, random_state=42
)

train_ds_sae = TensorDataset(
    torch.tensor(x_tr, dtype=torch.float32),
    torch.tensor(dx_tr, dtype=torch.float32),
    torch.tensor(ddx_tr, dtype=torch.float32)
)
val_ds_sae = TensorDataset(
    torch.tensor(x_vl, dtype=torch.float32),
    torch.tensor(dx_vl, dtype=torch.float32),
    torch.tensor(ddx_vl, dtype=torch.float32)
)

train_loader_sae = DataLoader(train_ds_sae, batch_size=256, shuffle=True)
val_loader_sae = DataLoader(val_ds_sae, batch_size=256, shuffle=False)

print(f"SINDy-Autoencoder data: {len(train_ds_sae)} train, {len(val_ds_sae)} val")

## 2.6 Training SINDy-Autoencoder

Train with:
- 5000 epochs + 1000 epochs refinement
- Sequential Thresholding: $a=0.1$, $S=500$ epochs
- Loss weights: $\lambda_{\ddot{z}}=5 \times 10^{-5}$, $\lambda_{\ddot{x}}=5 \times 10^{-4}$, $\lambda_{L1}=10^{-5}$

In [ ]:
# Train SINDy-Autoencoder with Sequential Thresholding
print("Training SINDy-Autoencoder with Sequential Thresholding...")
print("="*60)

sindy_ae_st = SINDyAutoencoder(
    input_dim=2, latent_dim=1,
    encoder_hidden=[32, 16], decoder_hidden=[16, 32],
    n_sindy_terms=10
).to(device)

history_sae_st = train_sindy_autoencoder(
    sindy_ae_st, train_loader_sae, val_loader_sae,
    n_epochs=6000,  # 5000 + 1000 refinement
    lr=1e-3,
    lambda_ddz=5e-5, lambda_ddx=5e-4, lambda_l1=1e-5,
    thresholding='sequential',
    threshold_a=0.1, threshold_interval=500,
    refinement_epoch=5000,
    verbose=True, log_interval=500
)

# Get final results
st_ae_coeffs = sindy_ae_st.get_sindy_coefficients()
st_ae_mask = sindy_ae_st.get_sindy_mask()

print("\n" + "="*60)
print("SINDy-Autoencoder (Sequential Thresholding) Results:")
print("-"*50)
for name, coef, mask in zip(term_names, st_ae_coeffs, st_ae_mask):
    status = "Active" if mask else "Inactive"
    print(f"  {name:15s}: {coef:10.6f} [{status}]")

### Visualize Training History

In [ ]:
def plot_sindy_ae_history(history, title, term_names, refinement_epoch=None):
    """Plot SINDy-Autoencoder training history."""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    epochs = np.arange(len(history['train_loss']))
    
    # Total loss
    axes[0, 0].semilogy(epochs, history['train_loss'], label='Train', alpha=0.8)
    axes[0, 0].semilogy(epochs, history['val_loss'], label='Val', alpha=0.8)
    if refinement_epoch:
        axes[0, 0].axvline(x=refinement_epoch, color='r', linestyle='--', label='Refinement')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Total Loss')
    axes[0, 0].set_title(f'{title} - Total Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Component losses
    axes[0, 1].semilogy(epochs, history['train_loss_x'], label='$L_x$', alpha=0.8)
    axes[0, 1].semilogy(epochs, history['train_loss_ddz'], label='$L_{\\ddot{z}}$', alpha=0.8)
    axes[0, 1].semilogy(epochs, history['train_loss_ddx'], label='$L_{\\ddot{x}}$', alpha=0.8)
    if refinement_epoch:
        axes[0, 1].axvline(x=refinement_epoch, color='r', linestyle='--', label='Refinement')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_title(f'{title} - Loss Components')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Coefficient evolution
    coeffs_history = np.array(history['coefficients'])
    for i, name in enumerate(term_names):
        axes[1, 0].plot(epochs, coeffs_history[:, i], label=name, alpha=0.8)
    if refinement_epoch:
        axes[1, 0].axvline(x=refinement_epoch, color='r', linestyle='--', label='Refinement')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Coefficient Value')
    axes[1, 0].set_title(f'{title} - Coefficient Evolution')
    axes[1, 0].legend(fontsize=8, loc='upper right')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Final coefficients
    final_coeffs = coeffs_history[-1]
    colors = ['green' if np.abs(c) > 0.05 else 'gray' for c in final_coeffs]
    bars = axes[1, 1].bar(term_names, final_coeffs, color=colors)
    axes[1, 1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    axes[1, 1].set_xlabel('Term')
    axes[1, 1].set_ylabel('Coefficient')
    axes[1, 1].set_title(f'{title} - Final Coefficients')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# Plot training history
plot_sindy_ae_history(history_sae_st, "Sequential Thresholding", term_names, refinement_epoch=5000)

### Train with Patient Trend-Aware Thresholding (PTAT)

In [ ]:
# Train SINDy-Autoencoder with PTAT
print("Training SINDy-Autoencoder with Patient Trend-Aware Thresholding...")
print("="*60)

sindy_ae_ptat = SINDyAutoencoder(
    input_dim=2, latent_dim=1,
    encoder_hidden=[32, 16], decoder_hidden=[16, 32],
    n_sindy_terms=10
).to(device)

history_sae_ptat = train_sindy_autoencoder(
    sindy_ae_ptat, train_loader_sae, val_loader_sae,
    n_epochs=6000,
    lr=1e-3,
    lambda_ddz=5e-5, lambda_ddx=5e-4, lambda_l1=1e-5,
    thresholding='patient',
    threshold_a=0.1, threshold_b=0.002, patience=1000,
    refinement_epoch=5000,
    verbose=True, log_interval=500
)

# Get final results
ptat_ae_coeffs = sindy_ae_ptat.get_sindy_coefficients()
ptat_ae_mask = sindy_ae_ptat.get_sindy_mask()

print("\n" + "="*60)
print("SINDy-Autoencoder (PTAT) Results:")
print("-"*50)
for name, coef, mask in zip(term_names, ptat_ae_coeffs, ptat_ae_mask):
    status = "Active" if mask else "Inactive"
    print(f"  {name:15s}: {coef:10.6f} [{status}]")

# Plot
plot_sindy_ae_history(history_sae_ptat, "PTAT", term_names, refinement_epoch=5000)

### Train Multiple Models for Statistical Analysis

In [ ]:
# Train 10 models with ST and PTAT for statistical analysis
# Note: This cell takes longer to run

n_models = 10
all_results_st = []
all_results_ptat = []

print(f"Training {n_models} models with each thresholding method...")
print("="*60)

for i in range(n_models):
    print(f"\n--- Model {i+1}/{n_models} ---")
    
    # Train ST model
    model_st = SINDyAutoencoder(
        input_dim=2, latent_dim=1,
        encoder_hidden=[32, 16], decoder_hidden=[16, 32],
        n_sindy_terms=10
    ).to(device)
    
    history_st = train_sindy_autoencoder(
        model_st, train_loader_sae, val_loader_sae,
        n_epochs=6000, lr=1e-3,
        lambda_ddz=5e-5, lambda_ddx=5e-4, lambda_l1=1e-5,
        thresholding='sequential',
        threshold_a=0.1, threshold_interval=500,
        refinement_epoch=5000,
        verbose=False
    )
    
    all_results_st.append({
        'coefficients': model_st.get_sindy_coefficients(),
        'mask': model_st.get_sindy_mask(),
        'model': model_st,
        'history': history_st
    })
    print(f"  ST: {model_st.get_sindy_mask().sum()} active terms")
    
    # Train PTAT model
    model_ptat = SINDyAutoencoder(
        input_dim=2, latent_dim=1,
        encoder_hidden=[32, 16], decoder_hidden=[16, 32],
        n_sindy_terms=10
    ).to(device)
    
    history_ptat = train_sindy_autoencoder(
        model_ptat, train_loader_sae, val_loader_sae,
        n_epochs=6000, lr=1e-3,
        lambda_ddz=5e-5, lambda_ddx=5e-4, lambda_l1=1e-5,
        thresholding='patient',
        threshold_a=0.1, threshold_b=0.002, patience=1000,
        refinement_epoch=5000,
        verbose=False
    )
    
    all_results_ptat.append({
        'coefficients': model_ptat.get_sindy_coefficients(),
        'mask': model_ptat.get_sindy_mask(),
        'model': model_ptat,
        'history': history_ptat
    })
    print(f"  PTAT: {model_ptat.get_sindy_mask().sum()} active terms")

print("\n" + "="*60)
print("Training complete!")

## 2.7 Evaluation & Visualization

### Compute FVU Metrics

Fraction of Variance Unexplained:
$$\text{FVU}_y = \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2}$$

In [ ]:
def compute_fvu(y_true, y_pred):
    """
    Compute Fraction of Variance Unexplained.
    
    FVU = Σ(y - ŷ)² / Σ(y - ȳ)²
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    
    return ss_res / ss_tot if ss_tot > 0 else 0.0


def evaluate_sindy_autoencoder(model, x, dx, ddx):
    """
    Evaluate SINDy-Autoencoder and compute FVU metrics.
    """
    model.eval()
    with torch.no_grad():
        x_t = torch.tensor(x, dtype=torch.float32).to(device)
        dx_t = torch.tensor(dx, dtype=torch.float32).to(device)
        ddx_t = torch.tensor(ddx, dtype=torch.float32).to(device)
        
        x_hat, ddz_hat, ddx_hat, z, dz, ddz = model(x_t, dx_t, ddx_t)
        
        x_hat = x_hat.cpu().numpy()
        ddz_hat = ddz_hat.cpu().numpy()
        ddx_hat = ddx_hat.cpu().numpy()
        ddz = ddz.cpu().numpy()
    
    # Compute FVUs
    fvu_x = compute_fvu(x.flatten(), x_hat.flatten())
    fvu_ddz = compute_fvu(ddz.flatten(), ddz_hat.flatten())
    fvu_ddx = compute_fvu(ddx.flatten(), ddx_hat.flatten())
    
    return {
        'fvu_x': fvu_x,
        'fvu_ddz': fvu_ddz,
        'fvu_ddx': fvu_ddx,
        'x_hat': x_hat,
        'ddz_hat': ddz_hat,
        'ddx_hat': ddx_hat
    }


# Evaluate all models
print("Evaluating models...")
print("="*60)

x_val = x_vl  # Validation data
dx_val = dx_vl
ddx_val = ddx_vl

fvu_results_st = []
fvu_results_ptat = []

for i, (r_st, r_ptat) in enumerate(zip(all_results_st, all_results_ptat)):
    eval_st = evaluate_sindy_autoencoder(r_st['model'], x_val, dx_val, ddx_val)
    eval_ptat = evaluate_sindy_autoencoder(r_ptat['model'], x_val, dx_val, ddx_val)
    
    fvu_results_st.append(eval_st)
    fvu_results_ptat.append(eval_ptat)

# Compute statistics
print("\nFVU Results (mean ± std):")
print("-"*50)

for name, results in [('ST', fvu_results_st), ('PTAT', fvu_results_ptat)]:
    fvu_x = [r['fvu_x'] for r in results]
    fvu_ddz = [r['fvu_ddz'] for r in results]
    fvu_ddx = [r['fvu_ddx'] for r in results]
    
    print(f"\n{name}:")
    print(f"  FVU_x:   {np.mean(fvu_x):.4f} ± {np.std(fvu_x):.4f}")
    print(f"  FVU_ddz: {np.mean(fvu_ddz):.4f} ± {np.std(fvu_ddz):.4f}")
    print(f"  FVU_ddx: {np.mean(fvu_ddx):.4f} ± {np.std(fvu_ddx):.4f}")

# Visualize FVU distributions
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

metrics = ['fvu_x', 'fvu_ddz', 'fvu_ddx']
titles = ['FVU$_x$', 'FVU$_{\\ddot{z}}$', 'FVU$_{\\ddot{x}}$']

for ax, metric, title in zip(axes, metrics, titles):
    st_vals = [r[metric] for r in fvu_results_st]
    ptat_vals = [r[metric] for r in fvu_results_ptat]
    
    x_pos = [1, 2]
    ax.boxplot([st_vals, ptat_vals], positions=x_pos)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['ST', 'PTAT'])
    ax.set_ylabel(title)
    ax.set_title(f'{title} Distribution')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Analyze Learned Equations

In [ ]:
# Analyze learned coefficients across models
def print_learned_equation(coefficients, mask, term_names, threshold=0.01):
    """Format learned equation as string."""
    active_terms = []
    for name, coef, m in zip(term_names, coefficients, mask):
        if m and np.abs(coef) > threshold:
            if coef > 0:
                active_terms.append(f"+{coef:.4f}·{name}")
            else:
                active_terms.append(f"{coef:.4f}·{name}")
    
    if active_terms:
        eq = " ".join(active_terms)
        if eq.startswith('+'):
            eq = eq[1:]
        return f"ẍ = {eq}"
    else:
        return "ẍ = 0"


print("Learned Equations Analysis:")
print("="*60)

print("\nSequential Thresholding (ST) Results:")
print("-"*50)
for i, r in enumerate(all_results_st):
    eq = print_learned_equation(r['coefficients'], r['mask'], term_names)
    n_active = r['mask'].sum()
    print(f"Model {i+1}: {eq} ({n_active} active terms)")

print("\nPatient Trend-Aware Thresholding (PTAT) Results:")
print("-"*50)
for i, r in enumerate(all_results_ptat):
    eq = print_learned_equation(r['coefficients'], r['mask'], term_names)
    n_active = r['mask'].sum()
    print(f"Model {i+1}: {eq} ({n_active} active terms)")

# Count which terms are most frequently selected
print("\n" + "="*60)
print("Term Selection Frequency:")
print("-"*50)

st_term_freq = np.zeros(len(term_names))
ptat_term_freq = np.zeros(len(term_names))

for r in all_results_st:
    for i, (coef, mask) in enumerate(zip(r['coefficients'], r['mask'])):
        if mask and np.abs(coef) > 0.01:
            st_term_freq[i] += 1

for r in all_results_ptat:
    for i, (coef, mask) in enumerate(zip(r['coefficients'], r['mask'])):
        if mask and np.abs(coef) > 0.01:
            ptat_term_freq[i] += 1

print(f"\n{'Term':15s} {'ST':>8s} {'PTAT':>8s}")
print("-"*35)
for name, st_f, ptat_f in zip(term_names, st_term_freq, ptat_term_freq):
    print(f"{name:15s} {int(st_f):>8d} {int(ptat_f):>8d}")

# Visualize term selection
fig, ax = plt.subplots(figsize=(12, 5))

x = np.arange(len(term_names))
width = 0.35

bars1 = ax.bar(x - width/2, st_term_freq / n_models * 100, width, label='ST', alpha=0.8)
bars2 = ax.bar(x + width/2, ptat_term_freq / n_models * 100, width, label='PTAT', alpha=0.8)

ax.set_xlabel('Term')
ax.set_ylabel('Selection Frequency (%)')
ax.set_title('Term Selection Frequency Across Models')
ax.set_xticks(x)
ax.set_xticklabels(term_names, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

### Resimulate with Learned Equations

In [ ]:
def resimulate_and_decode(sindy_ae_model, coefficients, terms, z0, dz0, T, dt):
    """
    Resimulate with learned coefficients and decode to x-space.
    """
    # Simulate in z-space
    t, z_sim, dz_sim, ddz_sim = simulate_pendulum(z0, dz0, coefficients, terms, T, dt)
    
    # Embed ground truth to x-space
    x_gt, dx_gt, ddx_gt = embed_cartesian(z_sim, dz_sim, ddz_sim)
    
    # Decode using trained decoder
    sindy_ae_model.eval()
    with torch.no_grad():
        z_t = torch.tensor(z_sim.reshape(-1, 1), dtype=torch.float32).to(device)
        x_decoded = sindy_ae_model.decode(z_t).cpu().numpy()
    
    return t, z_sim, x_gt, x_decoded


# Test resimulation with one of the best models
best_st_idx = np.argmin([r['fvu_x'] for r in fvu_results_st])
best_ptat_idx = np.argmin([r['fvu_x'] for r in fvu_results_ptat])

best_st_model = all_results_st[best_st_idx]['model']
best_ptat_model = all_results_ptat[best_ptat_idx]['model']

# Test initial conditions
test_ics = [(0.5, 0.0), (1.5, 0.5), (2.5, -0.3)]

fig, axes = plt.subplots(len(test_ics), 4, figsize=(16, 4*len(test_ics)))

for row, (z0, dz0) in enumerate(test_ics):
    # Ground truth
    t, z_gt, dz_gt, ddz_gt = simulate_pendulum(
        z0, dz0, GROUND_TRUTH_COEFFICIENTS, terms, 50, 0.02
    )
    x_gt, _, _ = embed_cartesian(z_gt, dz_gt, ddz_gt)
    
    # Resimulate with ST model
    t_st, z_st, x_gt_st, x_dec_st = resimulate_and_decode(
        best_st_model, all_results_st[best_st_idx]['coefficients'],
        terms, z0, dz0, 50, 0.02
    )
    
    # Resimulate with PTAT model
    t_ptat, z_ptat, x_gt_ptat, x_dec_ptat = resimulate_and_decode(
        best_ptat_model, all_results_ptat[best_ptat_idx]['coefficients'],
        terms, z0, dz0, 50, 0.02
    )
    
    # Plot z trajectory
    axes[row, 0].plot(t, z_gt, 'k-', label='Ground Truth', linewidth=2)
    axes[row, 0].plot(t_st, z_st, 'b--', label='ST', alpha=0.8)
    axes[row, 0].plot(t_ptat, z_ptat, 'r-.', label='PTAT', alpha=0.8)
    axes[row, 0].set_xlabel('Time (s)')
    axes[row, 0].set_ylabel('z')
    axes[row, 0].set_title(f'z(t): z₀={z0}, ż₀={dz0}')
    axes[row, 0].legend()
    axes[row, 0].grid(True, alpha=0.3)
    
    # Plot x trajectory
    axes[row, 1].plot(x_gt[:, 0], x_gt[:, 1], 'k-', label='Ground Truth', linewidth=2)
    axes[row, 1].plot(x_dec_st[:, 0], x_dec_st[:, 1], 'b--', label='ST Decoded', alpha=0.8)
    axes[row, 1].plot(x_dec_ptat[:, 0], x_dec_ptat[:, 1], 'r-.', label='PTAT Decoded', alpha=0.8)
    axes[row, 1].set_xlabel('x₁')
    axes[row, 1].set_ylabel('x₂')
    axes[row, 1].set_title('x(t) Decoded from z')
    axes[row, 1].legend()
    axes[row, 1].grid(True, alpha=0.3)
    axes[row, 1].set_aspect('equal')
    
    # Plot z error over time
    z_error_st = np.abs(z_st - z_gt)
    z_error_ptat = np.abs(z_ptat - z_gt)
    
    axes[row, 2].semilogy(t, z_error_st, 'b-', label='ST')
    axes[row, 2].semilogy(t, z_error_ptat, 'r-', label='PTAT')
    axes[row, 2].axvline(x=1.0, color='gray', linestyle='--', alpha=0.5)
    axes[row, 2].set_xlabel('Time (s)')
    axes[row, 2].set_ylabel('|z - ẑ|')
    axes[row, 2].set_title('z Error over Time')
    axes[row, 2].legend()
    axes[row, 2].grid(True, alpha=0.3)
    
    # Plot x error over time
    x_error_st = np.sqrt(np.sum((x_gt - x_dec_st)**2, axis=1))
    x_error_ptat = np.sqrt(np.sum((x_gt - x_dec_ptat)**2, axis=1))
    
    axes[row, 3].semilogy(t, x_error_st, 'b-', label='ST')
    axes[row, 3].semilogy(t, x_error_ptat, 'r-', label='PTAT')
    axes[row, 3].axvline(x=1.0, color='gray', linestyle='--', alpha=0.5)
    axes[row, 3].set_xlabel('Time (s)')
    axes[row, 3].set_ylabel('||x - x̂||')
    axes[row, 3].set_title('x Error over Time')
    axes[row, 3].legend()
    axes[row, 3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Note: Up to t=1s, resimulation should be very close to ground truth.")

---
# Part 3: Bonus - SINDy-Autoencoder on Videos

## 3.1 Artificial Video Embedding

Create video frames with a Gaussian peak at the pendulum tip location.

In [ ]:
def embed_grid(z, t, resolution=32, sigma=0.1):
    """
    Create video of a Gaussian peak at pendulum tip.
    
    Parameters:
    -----------
    z : ndarray
        Angle trajectory, shape (N, T) or (T,)
    t : ndarray
        Time array
    resolution : int
        Video frame resolution (resolution x resolution)
    sigma : float
        Gaussian standard deviation
    
    Returns:
    --------
    x : ndarray
        Video frames, shape (*z.shape, resolution*resolution)
    dx : ndarray
        Time derivative (finite difference)
    ddx : ndarray
        Second time derivative (finite difference)
    """
    z = np.atleast_2d(z)  # Shape: (N, T)
    N, T_steps = z.shape
    
    # Compute Cartesian coordinates of tip
    tip_x = np.sin(z)  # Shape: (N, T)
    tip_y = -np.cos(z)
    
    # Create pixel grid [-1.5, 1.5]
    pixel_coords = np.linspace(-1.5, 1.5, resolution)
    xx, yy = np.meshgrid(pixel_coords, pixel_coords)
    
    # Flatten grid for easier computation
    xx_flat = xx.flatten()  # Shape: (resolution*resolution,)
    yy_flat = yy.flatten()
    
    # Create video frames
    video = np.zeros((N, T_steps, resolution * resolution))
    
    for n in range(N):
        for t_idx in range(T_steps):
            # Compute Gaussian
            dx = xx_flat - tip_x[n, t_idx]
            dy = yy_flat - tip_y[n, t_idx]
            dist_sq = dx**2 + dy**2
            gaussian = np.exp(-dist_sq / (2 * sigma**2))
            video[n, t_idx] = gaussian
    
    # Compute derivatives using finite differences
    dt = t[1] - t[0] if len(t) > 1 else 0.02
    
    # First derivative: dx[t] = (x[t+1] - x[t-1]) / (2*dt)
    dx_video = np.zeros_like(video)
    dx_video[:, 1:-1] = (video[:, 2:] - video[:, :-2]) / (2 * dt)
    dx_video[:, 0] = (video[:, 1] - video[:, 0]) / dt
    dx_video[:, -1] = (video[:, -1] - video[:, -2]) / dt
    
    # Second derivative: ddx[t] = (x[t-1] - 2*x[t] + x[t+1]) / dt²
    ddx_video = np.zeros_like(video)
    ddx_video[:, 1:-1] = (video[:, :-2] - 2*video[:, 1:-1] + video[:, 2:]) / (dt**2)
    ddx_video[:, 0] = ddx_video[:, 1]
    ddx_video[:, -1] = ddx_video[:, -2]
    
    return video, dx_video, ddx_video


# Create video embedding for a sample trajectory
sample_z = train_data['z'][0:5]  # 5 trajectories
sample_t = train_data['t'][0]

video, dvideo, ddvideo = embed_grid(sample_z, sample_t, resolution=32, sigma=0.1)

print(f"Video embedding shapes:")
print(f"  Video: {video.shape}")
print(f"  dVideo: {dvideo.shape}")
print(f"  ddVideo: {ddvideo.shape}")

# Visualize video frames
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# First trajectory
traj_idx = 0
frames_to_show = [0, 10, 20, 30, 40]

for col, t_idx in enumerate(frames_to_show):
    frame = video[traj_idx, t_idx].reshape(32, 32)
    axes[0, col].imshow(frame, extent=[-1.5, 1.5, -1.5, 1.5], origin='lower', cmap='hot')
    axes[0, col].set_title(f't = {t_idx * 0.02:.2f}s')
    axes[0, col].set_xlabel('x₁')
    if col == 0:
        axes[0, col].set_ylabel('x₂')
    
    # Also show derivative frame
    dframe = dvideo[traj_idx, t_idx].reshape(32, 32)
    im = axes[1, col].imshow(dframe, extent=[-1.5, 1.5, -1.5, 1.5], origin='lower', cmap='RdBu')
    if col == 0:
        axes[1, col].set_ylabel('x₂')

axes[1, 2].set_xlabel('Video Derivative')

plt.suptitle('Video Embedding: Gaussian Peak at Pendulum Tip', fontsize=14)
plt.tight_layout()
plt.show()

## 3.2-3.3 SINDy-Autoencoder on Videos

Use encoder hidden layers [128, 64, 32] and decoder [32, 64, 128] as in Champion et al.

In [ ]:
# Create video data for training
print("Creating video training data...")
video_train, dvideo_train, ddvideo_train = embed_grid(
    train_data['z'], train_data['t'][0], resolution=32, sigma=0.1
)

# Flatten and reshape
video_flat = video_train.reshape(-1, 32*32)  # Shape: (N*T, 1024)
dvideo_flat = dvideo_train.reshape(-1, 32*32)
ddvideo_flat = ddvideo_train.reshape(-1, 32*32)

print(f"Video data shape: {video_flat.shape}")

# Split data
vid_tr, vid_vl, dvid_tr, dvid_vl, ddvid_tr, ddvid_vl = train_test_split(
    video_flat, dvideo_flat, ddvideo_flat, test_size=0.2, random_state=42
)

train_ds_vid = TensorDataset(
    torch.tensor(vid_tr, dtype=torch.float32),
    torch.tensor(dvid_tr, dtype=torch.float32),
    torch.tensor(ddvid_tr, dtype=torch.float32)
)
val_ds_vid = TensorDataset(
    torch.tensor(vid_vl, dtype=torch.float32),
    torch.tensor(dvid_vl, dtype=torch.float32),
    torch.tensor(ddvid_vl, dtype=torch.float32)
)

train_loader_vid = DataLoader(train_ds_vid, batch_size=256, shuffle=True)
val_loader_vid = DataLoader(val_ds_vid, batch_size=256, shuffle=False)

print(f"Video dataset: {len(train_ds_vid)} train, {len(val_ds_vid)} val")

### Train SINDy-Autoencoder on Video Data

In [ ]:
# Train SINDy-Autoencoder on video with ST
print("Training SINDy-Autoencoder on Video (Sequential Thresholding)...")
print("="*60)

sindy_ae_vid_st = SINDyAutoencoder(
    input_dim=32*32,  # Flattened video
    latent_dim=1,
    encoder_hidden=[128, 64, 32],
    decoder_hidden=[32, 64, 128],
    n_sindy_terms=10
).to(device)

history_vid_st = train_sindy_autoencoder(
    sindy_ae_vid_st, train_loader_vid, val_loader_vid,
    n_epochs=6000,
    lr=1e-3,
    lambda_ddz=5e-5, lambda_ddx=5e-4, lambda_l1=1e-5,
    thresholding='sequential',
    threshold_a=0.1, threshold_interval=500,
    refinement_epoch=5000,
    verbose=True, log_interval=500
)

vid_st_coeffs = sindy_ae_vid_st.get_sindy_coefficients()
vid_st_mask = sindy_ae_vid_st.get_sindy_mask()

print("\nVideo ST Results:")
print("-"*50)
for name, coef, mask in zip(term_names, vid_st_coeffs, vid_st_mask):
    if mask:
        print(f"  {name:15s}: {coef:10.6f} [Active]")

In [ ]:
# Train SINDy-Autoencoder on video with PTAT
print("Training SINDy-Autoencoder on Video (PTAT)...")
print("="*60)

sindy_ae_vid_ptat = SINDyAutoencoder(
    input_dim=32*32,
    latent_dim=1,
    encoder_hidden=[128, 64, 32],
    decoder_hidden=[32, 64, 128],
    n_sindy_terms=10
).to(device)

history_vid_ptat = train_sindy_autoencoder(
    sindy_ae_vid_ptat, train_loader_vid, val_loader_vid,
    n_epochs=6000,
    lr=1e-3,
    lambda_ddz=5e-5, lambda_ddx=5e-4, lambda_l1=1e-5,
    thresholding='patient',
    threshold_a=0.1, threshold_b=0.002, patience=1000,
    refinement_epoch=5000,
    verbose=True, log_interval=500
)

vid_ptat_coeffs = sindy_ae_vid_ptat.get_sindy_coefficients()
vid_ptat_mask = sindy_ae_vid_ptat.get_sindy_mask()

print("\nVideo PTAT Results:")
print("-"*50)
for name, coef, mask in zip(term_names, vid_ptat_coeffs, vid_ptat_mask):
    if mask:
        print(f"  {name:15s}: {coef:10.6f} [Active]")

## 3.4 Evaluation: Compare ST vs PTAT on Videos

In [ ]:
# Compare ST vs PTAT on video data
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot training histories
plot_sindy_ae_history(history_vid_st, "Video ST", term_names, refinement_epoch=5000)
plot_sindy_ae_history(history_vid_ptat, "Video PTAT", term_names, refinement_epoch=5000)

# Summary comparison
print("\n" + "="*60)
print("Video Data: ST vs PTAT Comparison")
print("="*60)

print("\nSequential Thresholding (ST):")
st_eq = print_learned_equation(vid_st_coeffs, vid_st_mask, term_names)
print(f"  Equation: {st_eq}")
print(f"  Active terms: {vid_st_mask.sum()}")

print("\nPatient Trend-Aware Thresholding (PTAT):")
ptat_eq = print_learned_equation(vid_ptat_coeffs, vid_ptat_mask, term_names)
print(f"  Equation: {ptat_eq}")
print(f"  Active terms: {vid_ptat_mask.sum()}")

# Compare coefficients
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x_pos = np.arange(len(term_names))
width = 0.35

# ST coefficients
colors_st = ['green' if vid_st_mask[i] else 'gray' for i in range(len(term_names))]
axes[0].bar(x_pos, vid_st_coeffs, color=colors_st)
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(term_names, rotation=45, ha='right')
axes[0].set_ylabel('Coefficient')
axes[0].set_title('Video ST - Final Coefficients')
axes[0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[0].grid(True, alpha=0.3)

# PTAT coefficients
colors_ptat = ['green' if vid_ptat_mask[i] else 'gray' for i in range(len(term_names))]
axes[1].bar(x_pos, vid_ptat_coeffs, color=colors_ptat)
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(term_names, rotation=45, ha='right')
axes[1].set_ylabel('Coefficient')
axes[1].set_title('Video PTAT - Final Coefficients')
axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("CONCLUSION")
print("="*60)
print("""
For high-dimensional data like videos:
- Sequential Thresholding (ST) may select incorrect or too many terms because
  it makes hard thresholding decisions at fixed intervals without considering
  the trend of coefficient changes.
  
- Patient Trend-Aware Thresholding (PTAT) is more robust because it:
  1. Monitors coefficient changes over time
  2. Only removes terms that have consistently small values AND small changes
  3. Uses a patience mechanism to avoid premature pruning

This demonstrates that while ST may work well for simpler problems (like 
Cartesian coordinates), more sophisticated thresholding like PTAT is needed
for challenging high-dimensional settings where the optimization landscape
is more complex.
""")

---
# Summary

This notebook implemented the SINDy (Sparse Identification of Nonlinear Dynamics) algorithm for symbolic regression:

## Key Accomplishments:

### Part 1: SINDy in Ground Truth Coordinates
- Implemented pendulum simulation with ground truth ODE: $\ddot{z} = -\sin(z)$
- Created SINDy library: $\Theta(z, \dot{z}) = [1, z, \dot{z}, \sin(z), z^2, ...]$
- Implemented LASSO regression using sklearn and PyTorch
- Developed Sequential Thresholding (ST) and Patient Trend-Aware Thresholding (PTAT)
- Demonstrated small angle approximation: $\sin(z) \approx z$

### Part 2: SINDy-Autoencoder
- Embedded pendulum data into 2D Cartesian coordinates
- Built autoencoder with derivative propagation through layers
- Implemented `SigmoidDerivatives` and `LinearDerivatives` layers
- Created `SINDyAutoencoder` combining representation learning with equation discovery
- Trained with refinement phase (L1 regularization → no regularization)
- Evaluated using Fraction of Variance Unexplained (FVU)

### Part 3: Bonus - Video Data
- Created artificial video embedding with Gaussian peaks
- Trained deeper networks for high-dimensional video input
- Demonstrated that PTAT is more reliable than ST for complex data

## Key Findings:
1. SINDy can correctly identify the pendulum equation $\ddot{z} = -\sin(z)$
2. Thresholding is essential for sparse coefficient selection
3. PTAT provides more robust coefficient selection than ST, especially for high-dimensional data
4. The autoencoder successfully learns to encode Cartesian/video data to the canonical angle coordinate